In [3]:

import os
import re
import json
import pandas as pd
import numpy as np
import czifile as zis

#empty dictionary
df={
    'Position_Name':[],
    'Cell_name':[],
    'Position_X':[],
    'Position_Y':[],
    'AcquisitionDate':[],
    'AcquisitionTime':[],
    'NChannel':[],
    'Nframes':[],
    'NZslices':[],
    'Tilerow':[],
    'Tilecolumn':[],
    'N_X':[],
    'N_Y':[],
    'Original_Filename':[],
    'Scale_X':[],
    'Scale_Y':[],
    'Scale_Z':[]
}



filepath='/Volumes/LoCelso/Secchi_Marine'

folder_list=['VWF_043_MS211118_onstage_plt_depletion']
remove_list=[]


for folder in folder_list:
    filepath=os.path.join(filepath,folder)

    for root, dirs, files in os.walk(filepath): # will open up all the folders, dirs is all the name of the folder it finds, files will contain all the filenames it finds
        for file in files:
            if file.endswith('.czi'):#need to be iterating through file
                duplicates=re.findall(".*(?=-LSM)",file)
                if len(duplicates)>1:
                    remove_list.append(duplicates[0]+'.czi') 
#                    print("test", file,duplicates[0])

        
        for file in files: 
            if file not in remove_list and file.endswith('.czi'):
                #print(file)
                filename=os.path.join(root,file)#looks at the directory used, joins the strings
                try:
                    czi = zis.CziFile(filename)
                except Exception as e:
                    print(f"{file} error {e}")
                    #print(filename)
                    # parse the XML into a dictionary
                metadatadict_czi = czi.metadata(raw=False)


                    

# For multiple positions per file wich should include movies and tilescans
                if "S" in metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys():
                    allscenes = metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['S']['Scenes']['Scene']
#                     if file == 'VWF_043_MS211125_m4-softwareautofocus_gfpref_1.czi':
#                         print(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image'].items())
                    if isinstance(allscenes, dict):
                        coordinates=allscenes['CenterPosition'].split(',')
                        df['Cell_name'].append(allscenes['Name'])
                        df['Position_Name'].append(file+'_'+allscenes['Name'])#changed from site to allscenes
                        df['Original_Filename'].append(file)
                        df['Position_X'].append(coordinates[0])
                        df['Position_Y'].append(coordinates[1])
                        dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                        df['NZslices'].append(dimensions['SizeZ']) 
                        df['NChannel'].append(dimensions['SizeC'])
                        df['Nframes'].append(dimensions['SizeT'])
                        df['N_X'].append(dimensions['SizeX'])
                        df['N_Y'].append(dimensions['SizeY'])
                        date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                        df['AcquisitionDate'].append(date[0])
                        df['AcquisitionTime'].append(date[1])
                        scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                        df['Scale_X'].append(scale['Value'])#Distance Id="X"
                        scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                        df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                        if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                            df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                        else:
                            df['Scale_Z'].append(1)#Distance Id="Z"

                        # try:
                        if "tilescan" in filename.casefold() and "RegionsSetup" in metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups'].keys(): 
                        #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            
                            tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                            if isinstance(tile, list):
                                for i in tile: #going inside the list !!!!
                                    #print(i)
                                    if i['IsUsedForAcquisition']==True:
                                        df['Tilecolumn'].append(i['Columns'])
                                        df['Tilerow'].append(i['Rows'])   
                                    else: 
                                        print(f'{file}IsUsedForAcquisition Not true')                                    

                            #will not work for single positions so the else is required
                            else:
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])
                
                        else:
                            df['Tilecolumn'].append('1')
                            df['Tilerow'].append('1')   
                        # except Exception as e:
                            # print(f"{file} error1 {e}")

                    else:
                            

                        for site in allscenes:#when scenes is a list you have to iterate through each element
                            #print(allscenes,file)
                            coordinates=site['CenterPosition'].split(',')
                            df['Cell_name'].append(site['Name'])
                            df['Position_Name'].append(file+'_'+site['Name'])
                            df['Original_Filename'].append(file)
                            df['Position_X'].append(coordinates[0])
                            df['Position_Y'].append(coordinates[1])
                            dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                            df['NZslices'].append(dimensions['SizeZ']) 
                            df['NChannel'].append(dimensions['SizeC'])
                            df['Nframes'].append(dimensions['SizeT'])
                            df['N_X'].append(dimensions['SizeX'])
                            df['N_Y'].append(dimensions['SizeY'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                            df['AcquisitionDate'].append(date[0])
                            df['AcquisitionTime'].append(date[1])
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                            df['Scale_X'].append(scale['Value'])#Distance Id="X"
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                            df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                            if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                                scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                                df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                            else:
                                df['Scale_Z'].append(1)#Distance Id="Z"

                            #try:
                            if "tilescan" in filename or "TILESCAN" in filename and "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                                tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])

                            else:
                                df['Tilecolumn'].append('1')
                                df['Tilerow'].append('1')   
                            #except Exception as e:
                            #    print(f"{file} error2")




#For single positions

                elif "HardwareSetting" in metadatadict_czi['ImageDocument']['Metadata'].keys() and "S" not in metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys():
                    
                    new_list=metadatadict_czi['ImageDocument']['Metadata']['HardwareSetting']['ParameterCollection']
                    for i in new_list: #i is every dict in the list 
#                         print(i)
                        if 'MTBStageAxisX' in i.values():
                            df['Cell_name'].append(file)
                            df['Position_Name'].append(file)
                            df['Original_Filename'].append(file)
#                             print(i['Position']['value'],i['Id'])
                            df['Position_X'].append(i['Position']['value'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                        if 'MTBStageAxisY' in i.values():
                            df['Position_Y'].append(i['Position']['value'])
                            dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                            df['NZslices'].append(dimensions['SizeZ']) 
                            df['NChannel'].append(dimensions['SizeC'])
                            df['Nframes'].append(dimensions['SizeT'])
                            df['N_X'].append(dimensions['SizeX'])
                            df['N_Y'].append(dimensions['SizeY'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                            df['AcquisitionDate'].append(date[0])
                            df['AcquisitionTime'].append(date[1])
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                            df['Scale_X'].append(scale['Value'])#Distance Id="X"
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                            df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                            if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                                scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                                df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                            else:
                                df['Scale_Z'].append(1)#Distance Id="Z"
                            #try:
                            if "tilescan" in filename or "TILESCAN" in filename and "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                                tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])

                            else:
                                df['Tilecolumn'].append('1')
                                df['Tilerow'].append('1')   
                        # except Exception as e:
                            #   print(f"{file} error3")

                    
                else:
                    print(file)

            
                

for key,value in df.items():
    print(key,len(df[key]))
pddf=pd.DataFrame(df)
#pddf=pddf.drop_duplicates(subset=['Position_Name'])
#pddf
#pddf=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in df.items() ]))#fills empty with na
pddf



._VWF_043_MS211126_m4-snap1699.czi error not a CZI file
._VWF_043_MS211126_m4-snap1694.czi error not a CZI file
._VWF_043_MS211126_m4-snap1689.czi error not a CZI file
._VWF_043_MS211126_m4-snap-1688.czi error not a CZI file
._VWF_043_MS211126_m4-snap-1687.czi error not a CZI file
._VWF_043_MS211126_m4-snap-1686.czi error not a CZI file
._VWF_043_MS211126_m4-snap-1685.czi error not a CZI file
._VWF_043_MS211126_m4-snap-1684.czi error not a CZI file
._VWF_043_MS211126_m4-snap-1683.czi error not a CZI file
._VWF_043_MS211126_m4-snap-1681.czi error not a CZI file
._VWF_043_MS211126_m4-snap-1680.czi error not a CZI file
._VWF_043_MS211126_m4-pltdepletion_new_position.czi error not a CZI file
._VWF_043_MS211126_m4-P20.czi error not a CZI file
._VWF_043_MS211126_m4-P19.czi error not a CZI file
._VWF_043_MS211126_m4-P17snap.czi error not a CZI file
._VWF_043_MS211126_m4-P17.czi error not a CZI file
._VWF_043_MS211126_m4-P1dp.czi error not a CZI file
._VWF_043_MS211126_m4-P12.czi error not a C

,Position_Name,Cell_name,Position_X,Position_Y,AcquisitionDate,AcquisitionTime,NChannel,Nframes,NZslices,Tilerow,Tilecolumn,N_X,N_Y,Original_Filename,Scale_X,Scale_Y,Scale_Z
0,Experiment-2209.czi_TR2,TR2,76601.363,44866.345,2021-11-25,17:14:51.4952653Z,4,1,6,1,1,973,513,Experiment-2209.czi,8.286426e-07,8.286426e-07,0.000002
1,Experiment-2210.czi_TR2,TR2,76601.363,44866.345,2021-11-25,17:17:36.299762Z,4,1,6,1,1,1433,512,Experiment-2210.czi,8.286426e-07,8.286426e-07,0.000002
2,Experiment-2211_bone_offset.czi_TR2,TR2,76601.363,44866.345,2021-11-25,17:25:23.1402856Z,4,1,6,1,1,2356,973,Experiment-2211_bone_offset.czi,8.286426e-07,8.286426e-07,0.000002
3,Experiment-2212_gfp_offset-padded-stiched.czi_TR2,TR2,76601.363,44866.345,2021-11-25,17:32:39.342351Z,4,1,44,1,1,2357,977,Experiment-2212_gfp_offset-padded-stiched.czi,8.286426e-07,8.286426e-07,0.000002
4,Experiment-2212_gfp_offset-padded.czi_TR2,TR2,76601.363,44866.345,2021-11-25,17:32:39.342351Z,4,1,44,1,1,2560,1024,Experiment-2212_gfp_offset-padded.czi,8.286426e-07,8.286426e-07,0.000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,VWF_043_MS211125_m3-tilescanpadded.czi_TR1,TR1,76597.781,44722.933,2021-11-25,11:10:35.175499Z,4,1,122,5,7,3584,2560,VWF_043_MS211125_m3-tilescanpadded.czi,8.286426e-07,8.286426e-07,0.000002
173,VWF_043_MS211126_m4-tilescan.czi_TR1,TR1,66297.882,45270.877,2021-11-26,18:43:58.5126641Z,4,1,28,4,7,3278,1895,VWF_043_MS211126_m4-tilescan.czi,8.286426e-07,8.286426e-07,0.000002
174,._VWF_043_MS211126_m4-tilescan.czi_TR1,TR1,66297.882,45270.877,2021-11-26,18:43:58.5126641Z,4,1,28,4,7,3278,1895,._VWF_043_MS211126_m4-tilescan.czi,8.286426e-07,8.286426e-07,0.000002
175,VWF_043_MS211126_m4-tilescanpadded.czi_TR1,TR1,66297.882,45270.877,2021-11-26,18:43:58.5126641Z,4,1,121,4,7,3584,2048,VWF_043_MS211126_m4-tilescanpadded.czi,8.286426e-07,8.286426e-07,0.000002


In [5]:
import os
import re
import json
import pandas as pd
import numpy as np
import czifile as zis

#empty dictionary
df={
    'Position_Name':[],
    'Cell_name':[],
    'Position_X':[],
    'Position_Y':[],
    'AcquisitionDate':[],
    'AcquisitionTime':[],
    'NChannel':[],
    'Nframes':[],
    'NZslices':[],
    'Tilerow':[],
    'Tilecolumn':[],
    'N_X':[],
    'N_Y':[],
    'Original_Filename':[],
    'Scale_X':[],
    'Scale_Y':[],
    'Scale_Z':[]
}



filepath='/Volumes/LoCelso/Secchi_Marine'

folder_list=['VWF_045_MS211208_2h_24h_pltdepletion_IVM']
remove_list=[]


for folder in folder_list:
    filepath=os.path.join(filepath,folder)

    for root, dirs, files in os.walk(filepath): # will open up all the folders, dirs is all the name of the folder it finds, files will contain all the filenames it finds
        for file in files:
            if file.endswith('.czi'):#need to be iterating through file
                duplicates=re.findall(".*(?=-LSM)",file)
                if len(duplicates)>1:
                    remove_list.append(duplicates[0]+'.czi') 
#                    print("test", file,duplicates[0])

        
        for file in files: 
            if file not in remove_list and file.endswith('.czi'):
                #print(file)
                filename=os.path.join(root,file)#looks at the directory used, joins the strings
                try:
                    czi = zis.CziFile(filename)
                except Exception as e:
                    print(f"{file} error {e}")
                    #print(filename)
                    # parse the XML into a dictionary
                metadatadict_czi = czi.metadata(raw=False)


                    

# For multiple positions per file wich should include movies and tilescans
                if "S" in metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys():
                    allscenes = metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['S']['Scenes']['Scene']
#                     if file == 'VWF_043_MS211125_m4-softwareautofocus_gfpref_1.czi':
#                         print(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image'].items())
                    if isinstance(allscenes, dict):
                        coordinates=allscenes['CenterPosition'].split(',')
                        df['Cell_name'].append(allscenes['Name'])
                        df['Position_Name'].append(file+'_'+allscenes['Name'])#changed from site to allscenes
                        df['Original_Filename'].append(file)
                        df['Position_X'].append(coordinates[0])
                        df['Position_Y'].append(coordinates[1])
                        dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                        df['NZslices'].append(dimensions['SizeZ']) 
                        df['NChannel'].append(dimensions['SizeC'])
                        df['Nframes'].append(dimensions['SizeT'])
                        df['N_X'].append(dimensions['SizeX'])
                        df['N_Y'].append(dimensions['SizeY'])
                        date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                        df['AcquisitionDate'].append(date[0])
                        df['AcquisitionTime'].append(date[1])
                        scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                        df['Scale_X'].append(scale['Value'])#Distance Id="X"
                        scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                        df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                        if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                            df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                        else:
                            df['Scale_Z'].append(1)#Distance Id="Z"

                        # try:
                        if "tilescan" in filename.casefold() and "RegionsSetup" in metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups'].keys(): 
                        #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            
                            tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                            if isinstance(tile, list):
                                for i in tile: #going inside the list !!!!
                                    #print(i)
                                    if i['IsUsedForAcquisition']==True:
                                        df['Tilecolumn'].append(i['Columns'])
                                        df['Tilerow'].append(i['Rows'])   
                                    else: 
                                        print(f'{file}IsUsedForAcquisition Not true')                                    

                            #will not work for single positions so the else is required
                            else:
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])
                
                        else:
                            df['Tilecolumn'].append('1')
                            df['Tilerow'].append('1')   
                        # except Exception as e:
                            # print(f"{file} error1 {e}")

                    else:
                            

                        for site in allscenes:#when scenes is a list you have to iterate through each element
                            #print(allscenes,file)
                            coordinates=site['CenterPosition'].split(',')
                            df['Cell_name'].append(site['Name'])
                            df['Position_Name'].append(file+'_'+site['Name'])
                            df['Original_Filename'].append(file)
                            df['Position_X'].append(coordinates[0])
                            df['Position_Y'].append(coordinates[1])
                            dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                            df['NZslices'].append(dimensions['SizeZ']) 
                            df['NChannel'].append(dimensions['SizeC'])
                            df['Nframes'].append(dimensions['SizeT'])
                            df['N_X'].append(dimensions['SizeX'])
                            df['N_Y'].append(dimensions['SizeY'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                            df['AcquisitionDate'].append(date[0])
                            df['AcquisitionTime'].append(date[1])
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                            df['Scale_X'].append(scale['Value'])#Distance Id="X"
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                            df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                            if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                                scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                                df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                            else:
                                df['Scale_Z'].append(1)#Distance Id="Z"

                            #try:
                            if "tilescan" in filename or "TILESCAN" in filename and "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                                tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])

                            else:
                                df['Tilecolumn'].append('1')
                                df['Tilerow'].append('1')   
                            #except Exception as e:
                            #    print(f"{file} error2")




#For single positions

                elif "HardwareSetting" in metadatadict_czi['ImageDocument']['Metadata'].keys() and "S" not in metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys():
                    
                    new_list=metadatadict_czi['ImageDocument']['Metadata']['HardwareSetting']['ParameterCollection']
                    for i in new_list: #i is every dict in the list 
#                         print(i)
                        if 'MTBStageAxisX' in i.values():
                            df['Cell_name'].append(file)
                            df['Position_Name'].append(file)
                            df['Original_Filename'].append(file)
#                             print(i['Position']['value'],i['Id'])
                            df['Position_X'].append(i['Position']['value'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                        if 'MTBStageAxisY' in i.values():
                            df['Position_Y'].append(i['Position']['value'])
                            dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                            df['NZslices'].append(dimensions['SizeZ']) 
                            df['NChannel'].append(dimensions['SizeC'])
                            df['Nframes'].append(dimensions['SizeT'])
                            df['N_X'].append(dimensions['SizeX'])
                            df['N_Y'].append(dimensions['SizeY'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                            df['AcquisitionDate'].append(date[0])
                            df['AcquisitionTime'].append(date[1])
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                            df['Scale_X'].append(scale['Value'])#Distance Id="X"
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                            df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                            if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                                scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                                df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                            else:
                                df['Scale_Z'].append(1)#Distance Id="Z"
                            #try:
                            if "tilescan" in filename or "TILESCAN" in filename and "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                                tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])

                            else:
                                df['Tilecolumn'].append('1')
                                df['Tilerow'].append('1')   
                        # except Exception as e:
                            #   print(f"{file} error3")

                    
                else:
                    print(file)

            
                

for key,value in df.items():
    print(key,len(df[key]))
pddf1=pd.DataFrame(df)
#pddf=pddf.drop_duplicates(subset=['Position_Name'])
#pddf
#pddf.to_csv('/Users/secchim/Documents/VWF_067_MS220909_metadata3.csv', index=False) #only works for dataframes #otherwise you get row numbers

#pddf=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in df.items() ]))#fills empty with na
pddf1



Position_Name 57
Cell_name 57
Position_X 57
Position_Y 57
AcquisitionDate 57
AcquisitionTime 57
NChannel 57
Nframes 57
NZslices 57
Tilerow 57
Tilecolumn 57
N_X 57
N_Y 57
Original_Filename 57
Scale_X 57
Scale_Y 57
Scale_Z 57


,Position_Name,Cell_name,Position_X,Position_Y,AcquisitionDate,AcquisitionTime,NChannel,Nframes,NZslices,Tilerow,Tilecolumn,N_X,N_Y,Original_Filename,Scale_X,Scale_Y,Scale_Z
0,VWF_045_MS211208_M2_p5.czi,VWF_045_MS211208_M2_p5.czi,9246.7,43942.3,2021-12-08,10:04:03.2551323Z,4,1,1,1,1,1024,1024,VWF_045_MS211208_M2_p5.czi,4.143213e-07,4.143213e-07,1.000000
1,VWF_045_MS211208_M2_p25.czi,VWF_045_MS211208_M2_p25.czi,8755.1,45174.7,2021-12-08,16:12:05.7965902Z,4,1,1,1,1,512,512,VWF_045_MS211208_M2_p25.czi,8.286426e-07,8.286426e-07,1.000000
2,VWF_045_MS211208_M2_p26.czi,VWF_045_MS211208_M2_p26.czi,9499.4,44796.3,2021-12-08,16:16:24.6226372Z,4,1,1,1,1,512,512,VWF_045_MS211208_M2_p26.czi,8.286426e-07,8.286426e-07,1.000000
3,VWF_045_MS211208_M2_p27.czi,VWF_045_MS211208_M2_p27.czi,11370.6,45550.3,2021-12-08,16:23:55.6657996Z,4,1,1,1,1,512,512,VWF_045_MS211208_M2_p27.czi,8.286426e-07,8.286426e-07,1.000000
4,VWF_045_MS211208_M2_tilescan.czi_TR1,TR1,10276.336,45197.964,2021-12-08,10:48:36.508029Z,4,1,29,5,9,4200,2354,VWF_045_MS211208_M2_tilescan.czi,8.286426e-07,8.286426e-07,0.000003
5,VWF_045_MS211209_M1_Snap-1839.czi,VWF_045_MS211209_M1_Snap-1839.czi,76329.3,44219.6,2021-12-09,09:57:03.2771541Z,4,1,1,1,1,512,512,VWF_045_MS211209_M1_Snap-1839.czi,8.286426e-07,8.286426e-07,1.000000
6,VWF_045_MS211209_M1_Snap-1841.czi,VWF_045_MS211209_M1_Snap-1841.czi,76498.2,44619.2,2021-12-09,10:00:30.9670192Z,4,1,1,1,1,512,512,VWF_045_MS211209_M1_Snap-1841.czi,8.286426e-07,8.286426e-07,1.000000
7,VWF_045_MS211209_M1_Snap-1840.czi,VWF_045_MS211209_M1_Snap-1840.czi,76329.3,44486.3,2021-12-09,09:58:28.5870581Z,4,1,1,1,1,512,512,VWF_045_MS211209_M1_Snap-1840.czi,8.286426e-07,8.286426e-07,1.000000
8,VWF_045_MS211209_M1_Snap-1842.czi,VWF_045_MS211209_M1_Snap-1842.czi,76506.3,44273.8,2021-12-09,10:02:05.3839554Z,4,1,1,1,1,512,512,VWF_045_MS211209_M1_Snap-1842.czi,8.286426e-07,8.286426e-07,1.000000
9,VWF_045_MS211209_M1_Snap-1843.czi,VWF_045_MS211209_M1_Snap-1843.czi,76707.8,44691.3,2021-12-09,10:04:15.5226195Z,4,1,1,1,1,512,512,VWF_045_MS211209_M1_Snap-1843.czi,8.286426e-07,8.286426e-07,1.000000


In [6]:

import os
import re
import json
import pandas as pd
import numpy as np
import czifile as zis

#empty dictionary
df={
    'Position_Name':[],
    'Cell_name':[],
    'Position_X':[],
    'Position_Y':[],
    'AcquisitionDate':[],
    'AcquisitionTime':[],
    'NChannel':[],
    'Nframes':[],
    'NZslices':[],
    'Tilerow':[],
    'Tilecolumn':[],
    'N_X':[],
    'N_Y':[],
    'Original_Filename':[],
    'Scale_X':[],
    'Scale_Y':[],
    'Scale_Z':[]
}



filepath='/Volumes/LoCelso/Secchi_Marine'

folder_list=['VWF_048_MS220117_IVM_depletion']
remove_list=[]


for folder in folder_list:
    filepath=os.path.join(filepath,folder)

    for root, dirs, files in os.walk(filepath): # will open up all the folders, dirs is all the name of the folder it finds, files will contain all the filenames it finds
        for file in files:
            if file.endswith('.czi'):#need to be iterating through file
                duplicates=re.findall(".*(?=-LSM)",file)
                if len(duplicates)>1:
                    remove_list.append(duplicates[0]+'.czi') 
#                    print("test", file,duplicates[0])

        
        for file in files: 
            if file not in remove_list and file.endswith('.czi'):
                #print(file)
                filename=os.path.join(root,file)#looks at the directory used, joins the strings
                try:
                    czi = zis.CziFile(filename)
                except Exception as e:
                    print(f"{file} error {e}")
                    #print(filename)
                    # parse the XML into a dictionary
                metadatadict_czi = czi.metadata(raw=False)


                    

# For multiple positions per file wich should include movies and tilescans
                if "S" in metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys():
                    allscenes = metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['S']['Scenes']['Scene']
#                     if file == 'VWF_043_MS211125_m4-softwareautofocus_gfpref_1.czi':
#                         print(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image'].items())
                    if isinstance(allscenes, dict):
                        coordinates=allscenes['CenterPosition'].split(',')
                        df['Cell_name'].append(allscenes['Name'])
                        df['Position_Name'].append(file+'_'+allscenes['Name'])#changed from site to allscenes
                        df['Original_Filename'].append(file)
                        df['Position_X'].append(coordinates[0])
                        df['Position_Y'].append(coordinates[1])
                        dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                        df['NZslices'].append(dimensions['SizeZ']) 
                        df['NChannel'].append(dimensions['SizeC'])
                        df['Nframes'].append(dimensions['SizeT'])
                        df['N_X'].append(dimensions['SizeX'])
                        df['N_Y'].append(dimensions['SizeY'])
                        date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                        df['AcquisitionDate'].append(date[0])
                        df['AcquisitionTime'].append(date[1])
                        scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                        df['Scale_X'].append(scale['Value'])#Distance Id="X"
                        scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                        df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                        if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                            df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                        else:
                            df['Scale_Z'].append(1)#Distance Id="Z"

                        # try:
                        if "tilescan" in filename.casefold() and "RegionsSetup" in metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups'].keys(): 
                        #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            
                            tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                            if isinstance(tile, list):
                                for i in tile: #going inside the list !!!!
                                    #print(i)
                                    if i['IsUsedForAcquisition']==True:
                                        df['Tilecolumn'].append(i['Columns'])
                                        df['Tilerow'].append(i['Rows'])   
                                    else: 
                                        print(f'{file}IsUsedForAcquisition Not true')                                    

                            #will not work for single positions so the else is required
                            else:
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])
                
                        else:
                            df['Tilecolumn'].append('1')
                            df['Tilerow'].append('1')   
                        # except Exception as e:
                            # print(f"{file} error1 {e}")

                    else:
                            

                        for site in allscenes:#when scenes is a list you have to iterate through each element
                            #print(allscenes,file)
                            coordinates=site['CenterPosition'].split(',')
                            df['Cell_name'].append(site['Name'])
                            df['Position_Name'].append(file+'_'+site['Name'])
                            df['Original_Filename'].append(file)
                            df['Position_X'].append(coordinates[0])
                            df['Position_Y'].append(coordinates[1])
                            dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                            df['NZslices'].append(dimensions['SizeZ']) 
                            df['NChannel'].append(dimensions['SizeC'])
                            df['Nframes'].append(dimensions['SizeT'])
                            df['N_X'].append(dimensions['SizeX'])
                            df['N_Y'].append(dimensions['SizeY'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                            df['AcquisitionDate'].append(date[0])
                            df['AcquisitionTime'].append(date[1])
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                            df['Scale_X'].append(scale['Value'])#Distance Id="X"
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                            df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                            if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                                scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                                df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                            else:
                                df['Scale_Z'].append(1)#Distance Id="Z"

                            #try:
                            if "tilescan" in filename or "TILESCAN" in filename and "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                                tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])

                            else:
                                df['Tilecolumn'].append('1')
                                df['Tilerow'].append('1')   
                            #except Exception as e:
                            #    print(f"{file} error2")




#For single positions

                elif "HardwareSetting" in metadatadict_czi['ImageDocument']['Metadata'].keys() and "S" not in metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys():
                    
                    new_list=metadatadict_czi['ImageDocument']['Metadata']['HardwareSetting']['ParameterCollection']
                    for i in new_list: #i is every dict in the list 
#                         print(i)
                        if 'MTBStageAxisX' in i.values():
                            df['Cell_name'].append(file)
                            df['Position_Name'].append(file)
                            df['Original_Filename'].append(file)
#                             print(i['Position']['value'],i['Id'])
                            df['Position_X'].append(i['Position']['value'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                        if 'MTBStageAxisY' in i.values():
                            df['Position_Y'].append(i['Position']['value'])
                            dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                            df['NZslices'].append(dimensions['SizeZ']) 
                            df['NChannel'].append(dimensions['SizeC'])
                            df['Nframes'].append(dimensions['SizeT'])
                            df['N_X'].append(dimensions['SizeX'])
                            df['N_Y'].append(dimensions['SizeY'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                            df['AcquisitionDate'].append(date[0])
                            df['AcquisitionTime'].append(date[1])
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                            df['Scale_X'].append(scale['Value'])#Distance Id="X"
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                            df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                            if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                                scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                                df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                            else:
                                df['Scale_Z'].append(1)#Distance Id="Z"
                            #try:
                            if "tilescan" in filename or "TILESCAN" in filename and "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                                tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])

                            else:
                                df['Tilecolumn'].append('1')
                                df['Tilerow'].append('1')   
                        # except Exception as e:
                            #   print(f"{file} error3")

                    
                else:
                    print(file)

            
                

for key,value in df.items():
    print(key,len(df[key]))
pddf2=pd.DataFrame(df)
#pddf=pddf.drop_duplicates(subset=['Position_Name'])
#pddf
#pddf.to_csv('/Users/secchim/Documents/VWF_067_MS220909_metadata3.csv', index=False) #only works for dataframes #otherwise you get row numbers

#pddf=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in df.items() ]))#fills empty with na
pddf2



VWF_048_MS220120_M2_tilescan_depletion2-LSM Plus Processing-03.cziIsUsedForAcquisition Not true
VWF_048_MS220120_M2_tilescan_depletion-LSM Plus Processing-02padded.cziIsUsedForAcquisition Not true
VWF_048_MS220120_M2_tilescan_depletion-LSM Plus Processing-02.cziIsUsedForAcquisition Not true
Position_Name 286
Cell_name 286
Position_X 286
Position_Y 286
AcquisitionDate 286
AcquisitionTime 286
NChannel 286
Nframes 286
NZslices 286
Tilerow 286
Tilecolumn 286
N_X 286
N_Y 286
Original_Filename 286
Scale_X 286
Scale_Y 286
Scale_Z 286


,Position_Name,Cell_name,Position_X,Position_Y,AcquisitionDate,AcquisitionTime,NChannel,Nframes,NZslices,Tilerow,Tilecolumn,N_X,N_Y,Original_Filename,Scale_X,Scale_Y,Scale_Z
0,VWF_048_MS220118_M3_P2.czi,VWF_048_MS220118_M3_P2.czi,1215.2,-13823.4,2022-01-18,14:37:40.7214065Z,3,1,1,1,1,1024,1024,VWF_048_MS220118_M3_P2.czi,2.437184e-07,2.437184e-07,1.000000
1,VWF_048_MS220118_M3_P26.czi,VWF_048_MS220118_M3_P26.czi,1373.7,-13428.5,2022-01-18,14:44:00.8672456Z,3,1,1,1,1,1024,1024,VWF_048_MS220118_M3_P26.czi,2.437184e-07,2.437184e-07,1.000000
2,VWF_048_MS220118_M3_P6.czi,VWF_048_MS220118_M3_P6.czi,1391.2,-13350.6,2022-01-18,14:46:00.4485121Z,3,1,1,1,1,1024,1024,VWF_048_MS220118_M3_P6.czi,2.437184e-07,2.437184e-07,1.000000
3,Snap-2006.czi,Snap-2006.czi,1748.4,-13884,2022-01-18,12:24:57.2716334Z,3,1,1,1,1,512,512,Snap-2006.czi,8.286426e-07,8.286426e-07,1.000000
4,VWF_048_MS220118_M3_P5.czi,VWF_048_MS220118_M3_P5.czi,1426.4,-13483,2022-01-18,14:53:30.9865025Z,3,1,1,1,1,1024,1024,VWF_048_MS220118_M3_P5.czi,2.301785e-07,2.301785e-07,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,VWF_048_MS220120_M2_P7zstack.czi_P7,P7,64260.6,45147.3,2022-01-20,09:52:55.7268049Z,3,1,4,1,1,1168,1168,VWF_048_MS220120_M2_P7zstack.czi,2.136709e-07,2.136709e-07,0.000003
282,VWF_048_MS220118_M2_P17_3cellszstack.czi_P173c...,P173cells+,2065.1,-12848.3,2022-01-18,13:05:38.5180742Z,3,1,12,1,1,1024,1024,VWF_048_MS220118_M2_P17_3cellszstack.czi,2.437184e-07,2.437184e-07,0.000003
283,VWF_048_MS220118_M2_P12_3cellszstackpost movie...,P12+++,2606.5,-12865,2022-01-18,13:08:01.0547847Z,3,1,12,1,1,1024,1024,VWF_048_MS220118_M2_P12_3cellszstackpost movie...,2.437184e-07,2.437184e-07,0.000003
284,VWF_048_MS220119_M1_pltdepletion_tilescan-LSM ...,TR1,51945.077,45139.256,2022-01-19,16:10:13.220768Z,4,1,14,4,9,16800,7580,VWF_048_MS220119_M1_pltdepletion_tilescan-LSM ...,2.071606e-07,2.071606e-07,0.000003


In [7]:

import os
import re
import json
import pandas as pd
import numpy as np
import czifile as zis

#empty dictionary
df={
    'Position_Name':[],
    'Cell_name':[],
    'Position_X':[],
    'Position_Y':[],
    'AcquisitionDate':[],
    'AcquisitionTime':[],
    'NChannel':[],
    'Nframes':[],
    'NZslices':[],
    'Tilerow':[],
    'Tilecolumn':[],
    'N_X':[],
    'N_Y':[],
    'Original_Filename':[],
    'Scale_X':[],
    'Scale_Y':[],
    'Scale_Z':[]
}


filepath='/Volumes/LoCelso/Secchi_Marine'

folder_list=['VWF_049_MS220201_IVM_depletion']
remove_list=[]


for folder in folder_list:
    filepath=os.path.join(filepath,folder)

    for root, dirs, files in os.walk(filepath): # will open up all the folders, dirs is all the name of the folder it finds, files will contain all the filenames it finds
        for file in files:
            if file.endswith('.czi'):#need to be iterating through file
                duplicates=re.findall(".*(?=-LSM)",file)
                if len(duplicates)>1:
                    remove_list.append(duplicates[0]+'.czi') 
#                    print("test", file,duplicates[0])

        
        for file in files: 
            if file not in remove_list and file.endswith('.czi'):
                #print(file)
                filename=os.path.join(root,file)#looks at the directory used, joins the strings
                try:
                    czi = zis.CziFile(filename)
                except Exception as e:
                    print(f"{file} error {e}")
                    #print(filename)
                    # parse the XML into a dictionary
                metadatadict_czi = czi.metadata(raw=False)


                    

# For multiple positions per file wich should include movies and tilescans
                if "S" in metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys():
                    allscenes = metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['S']['Scenes']['Scene']
#                     if file == 'VWF_043_MS211125_m4-softwareautofocus_gfpref_1.czi':
#                         print(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image'].items())
                    if isinstance(allscenes, dict):
                        coordinates=allscenes['CenterPosition'].split(',')
                        df['Cell_name'].append(allscenes['Name'])
                        df['Position_Name'].append(file+'_'+allscenes['Name'])#changed from site to allscenes
                        df['Original_Filename'].append(file)
                        df['Position_X'].append(coordinates[0])
                        df['Position_Y'].append(coordinates[1])
                        dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                        df['NZslices'].append(dimensions['SizeZ']) 
                        df['NChannel'].append(dimensions['SizeC'])
                        df['Nframes'].append(dimensions['SizeT'])
                        df['N_X'].append(dimensions['SizeX'])
                        df['N_Y'].append(dimensions['SizeY'])
                        date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                        df['AcquisitionDate'].append(date[0])
                        df['AcquisitionTime'].append(date[1])
                        scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                        df['Scale_X'].append(scale['Value'])#Distance Id="X"
                        scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                        df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                        if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                            df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                        else:
                            df['Scale_Z'].append(1)#Distance Id="Z"

                        # try:
                        if "tilescan" in filename.casefold() and "RegionsSetup" in metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups'].keys(): 
                        #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            
                            tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                            if isinstance(tile, list):
                                for i in tile: #going inside the list !!!!
                                    #print(i)
                                    if i['IsUsedForAcquisition']==True:
                                        df['Tilecolumn'].append(i['Columns'])
                                        df['Tilerow'].append(i['Rows'])   
                                    else: 
                                        print(f'{file}IsUsedForAcquisition Not true')                                    

                            #will not work for single positions so the else is required
                            else:
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])
                
                        else:
                            df['Tilecolumn'].append('1')
                            df['Tilerow'].append('1')   
                        # except Exception as e:
                            # print(f"{file} error1 {e}")

                    else:
                            

                        for site in allscenes:#when scenes is a list you have to iterate through each element
                            #print(allscenes,file)
                            coordinates=site['CenterPosition'].split(',')
                            df['Cell_name'].append(site['Name'])
                            df['Position_Name'].append(file+'_'+site['Name'])
                            df['Original_Filename'].append(file)
                            df['Position_X'].append(coordinates[0])
                            df['Position_Y'].append(coordinates[1])
                            dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                            df['NZslices'].append(dimensions['SizeZ']) 
                            df['NChannel'].append(dimensions['SizeC'])
                            df['Nframes'].append(dimensions['SizeT'])
                            df['N_X'].append(dimensions['SizeX'])
                            df['N_Y'].append(dimensions['SizeY'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                            df['AcquisitionDate'].append(date[0])
                            df['AcquisitionTime'].append(date[1])
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                            df['Scale_X'].append(scale['Value'])#Distance Id="X"
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                            df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                            if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                                scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                                df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                            else:
                                df['Scale_Z'].append(1)#Distance Id="Z"

                            #try:
                            if "tilescan" in filename or "TILESCAN" in filename and "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                                tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])

                            else:
                                df['Tilecolumn'].append('1')
                                df['Tilerow'].append('1')   
                            #except Exception as e:
                            #    print(f"{file} error2")




#For single positions

                elif "HardwareSetting" in metadatadict_czi['ImageDocument']['Metadata'].keys() and "S" not in metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys():
                    
                    new_list=metadatadict_czi['ImageDocument']['Metadata']['HardwareSetting']['ParameterCollection']
                    for i in new_list: #i is every dict in the list 
#                         print(i)
                        if 'MTBStageAxisX' in i.values():
                            df['Cell_name'].append(file)
                            df['Position_Name'].append(file)
                            df['Original_Filename'].append(file)
#                             print(i['Position']['value'],i['Id'])
                            df['Position_X'].append(i['Position']['value'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                        if 'MTBStageAxisY' in i.values():
                            df['Position_Y'].append(i['Position']['value'])
                            dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                            df['NZslices'].append(dimensions['SizeZ']) 
                            df['NChannel'].append(dimensions['SizeC'])
                            df['Nframes'].append(dimensions['SizeT'])
                            df['N_X'].append(dimensions['SizeX'])
                            df['N_Y'].append(dimensions['SizeY'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                            df['AcquisitionDate'].append(date[0])
                            df['AcquisitionTime'].append(date[1])
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                            df['Scale_X'].append(scale['Value'])#Distance Id="X"
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                            df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                            if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                                scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                                df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                            else:
                                df['Scale_Z'].append(1)#Distance Id="Z"
                            #try:
                            if "tilescan" in filename or "TILESCAN" in filename and "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                                tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])

                            else:
                                df['Tilecolumn'].append('1')
                                df['Tilerow'].append('1')   
                        # except Exception as e:
                            #   print(f"{file} error3")

                    
                else:
                    print(file)

            
                

for key,value in df.items():
    print(key,len(df[key]))
pddf3=pd.DataFrame(df)
#pddf=pddf.drop_duplicates(subset=['Position_Name'])
#pddf
#pddf.to_csv('/Users/secchim/Documents/VWF_067_MS220909_metadata3.csv', index=False) #only works for dataframes #otherwise you get row numbers

#pddf=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in df.items() ]))#fills empty with na
pddf3



Position_Name 128
Cell_name 128
Position_X 128
Position_Y 128
AcquisitionDate 128
AcquisitionTime 128
NChannel 128
Nframes 128
NZslices 128
Tilerow 128
Tilecolumn 128
N_X 128
N_Y 128
Original_Filename 128
Scale_X 128
Scale_Y 128
Scale_Z 128


,Position_Name,Cell_name,Position_X,Position_Y,AcquisitionDate,AcquisitionTime,NChannel,Nframes,NZslices,Tilerow,Tilecolumn,N_X,N_Y,Original_Filename,Scale_X,Scale_Y,Scale_Z
0,VWF_049_MS220201_M1_Snap_afterdepletion.czi,VWF_049_MS220201_M1_Snap_afterdepletion.czi,5248,-6643.2,2022-02-01,15:04:22.9975504Z,4,1,1,1,1,512,512,VWF_049_MS220201_M1_Snap_afterdepletion.czi,8.286426e-07,8.286426e-07,1.000000
1,VWF_049_MS220201_M1_Snap_P1dpltotherkuo.czi,VWF_049_MS220201_M1_Snap_P1dpltotherkuo.czi,6590.8,-5770.5,2022-02-01,17:49:35.7248832Z,4,1,1,1,1,1168,1168,VWF_049_MS220201_M1_Snap_P1dpltotherkuo.czi,2.136709e-07,2.136709e-07,1.000000
2,VWF_049_MS220201_M1_Snap_P2-LSM Plus Processin...,VWF_049_MS220201_M1_Snap_P2-LSM Plus Processin...,4990.6,-6906.9,2022-02-01,12:31:56.4497358Z,3,1,1,1,1,1168,1168,VWF_049_MS220201_M1_Snap_P2-LSM Plus Processin...,2.136709e-07,2.136709e-07,1.000000
3,VWF_049_MS220201_M1_Snap_P2_2-LSM Plus Process...,VWF_049_MS220201_M1_Snap_P2_2-LSM Plus Process...,4990.3,-6907.2,2022-02-01,12:34:22.6736747Z,3,1,1,1,1,1168,1168,VWF_049_MS220201_M1_Snap_P2_2-LSM Plus Process...,2.136709e-07,2.136709e-07,1.000000
4,VWF_049_MS220201_M1_Snap_P3-LSM Plus Processin...,VWF_049_MS220201_M1_Snap_P3-LSM Plus Processin...,5191.5,-6636.2,2022-02-01,12:35:15.8370574Z,3,1,1,1,1,1168,1168,VWF_049_MS220201_M1_Snap_P3-LSM Plus Processin...,2.136709e-07,2.136709e-07,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,VWF_049_MS220201_M1_Snap_P10otherZ1x-LSM Plus ...,P10spot,7177.6,-6701.5,2022-02-01,13:10:23.7015815Z,3,1,10,1,1,512,512,VWF_049_MS220201_M1_Snap_P10otherZ1x-LSM Plus ...,8.286426e-07,8.286426e-07,0.000003
124,VWF_049_MS220201_M1_Snap_P10otherZ1x-LSM Plus ...,P9kuo2,6915.4,-6506.3,2022-02-01,13:10:23.7015815Z,3,1,10,1,1,512,512,VWF_049_MS220201_M1_Snap_P10otherZ1x-LSM Plus ...,8.286426e-07,8.286426e-07,0.000003
125,VWF_049_MS220201_M1_Snap_P10otherZ1x-LSM Plus ...,P12DP,6982.7,-5923,2022-02-01,13:10:23.7015815Z,3,1,10,1,1,512,512,VWF_049_MS220201_M1_Snap_P10otherZ1x-LSM Plus ...,8.286426e-07,8.286426e-07,0.000003
126,VWF_049_MS220201_M1_Snap_P10otherZ1x-LSM Plus ...,P14touchMK,6126.1,-5691.2,2022-02-01,13:10:23.7015815Z,3,1,10,1,1,512,512,VWF_049_MS220201_M1_Snap_P10otherZ1x-LSM Plus ...,8.286426e-07,8.286426e-07,0.000003


In [8]:

import os
import re
import json
import pandas as pd
import numpy as np
import czifile as zis

#empty dictionary
df={
    'Position_Name':[],
    'Cell_name':[],
    'Position_X':[],
    'Position_Y':[],
    'AcquisitionDate':[],
    'AcquisitionTime':[],
    'NChannel':[],
    'Nframes':[],
    'NZslices':[],
    'Tilerow':[],
    'Tilecolumn':[],
    'N_X':[],
    'N_Y':[],
    'Original_Filename':[],
    'Scale_X':[],
    'Scale_Y':[],
    'Scale_Z':[]
}



filepath='/Volumes/LoCelso/Secchi_Marine'

folder_list=['VWF_053_MS220208_IVMdepletion']
remove_list=[]


for folder in folder_list:
    filepath=os.path.join(filepath,folder)

    for root, dirs, files in os.walk(filepath): # will open up all the folders, dirs is all the name of the folder it finds, files will contain all the filenames it finds
        for file in files:
            if file.endswith('.czi'):#need to be iterating through file
                duplicates=re.findall(".*(?=-LSM)",file)
                if len(duplicates)>1:
                    remove_list.append(duplicates[0]+'.czi') 
#                    print("test", file,duplicates[0])

        
        for file in files: 
            if file not in remove_list and file.endswith('.czi'):
                #print(file)
                filename=os.path.join(root,file)#looks at the directory used, joins the strings
                try:
                    czi = zis.CziFile(filename)
                except Exception as e:
                    print(f"{file} error {e}")
                    #print(filename)
                    # parse the XML into a dictionary
                metadatadict_czi = czi.metadata(raw=False)


                    

# For multiple positions per file wich should include movies and tilescans
                if "S" in metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys():
                    allscenes = metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['S']['Scenes']['Scene']
#                     if file == 'VWF_043_MS211125_m4-softwareautofocus_gfpref_1.czi':
#                         print(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image'].items())
                    if isinstance(allscenes, dict):
                        coordinates=allscenes['CenterPosition'].split(',')
                        df['Cell_name'].append(allscenes['Name'])
                        df['Position_Name'].append(file+'_'+allscenes['Name'])#changed from site to allscenes
                        df['Original_Filename'].append(file)
                        df['Position_X'].append(coordinates[0])
                        df['Position_Y'].append(coordinates[1])
                        dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                        df['NZslices'].append(dimensions['SizeZ']) 
                        df['NChannel'].append(dimensions['SizeC'])
                        df['Nframes'].append(dimensions['SizeT'])
                        df['N_X'].append(dimensions['SizeX'])
                        df['N_Y'].append(dimensions['SizeY'])
                        date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                        df['AcquisitionDate'].append(date[0])
                        df['AcquisitionTime'].append(date[1])
                        scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                        df['Scale_X'].append(scale['Value'])#Distance Id="X"
                        scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                        df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                        if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                            df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                        else:
                            df['Scale_Z'].append(1)#Distance Id="Z"

                        # try:
                        if "tilescan" in filename.casefold() and "RegionsSetup" in metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups'].keys(): 
                        #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            
                            tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                            if isinstance(tile, list):
                                for i in tile: #going inside the list !!!!
                                    #print(i)
                                    if i['IsUsedForAcquisition']==True:
                                        df['Tilecolumn'].append(i['Columns'])
                                        df['Tilerow'].append(i['Rows'])   
                                    else: 
                                        print(f'{file}IsUsedForAcquisition Not true')                                    

                            #will not work for single positions so the else is required
                            else:
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])
                
                        else:
                            df['Tilecolumn'].append('1')
                            df['Tilerow'].append('1')   
                        # except Exception as e:
                            # print(f"{file} error1 {e}")

                    else:
                            

                        for site in allscenes:#when scenes is a list you have to iterate through each element
                            #print(allscenes,file)
                            coordinates=site['CenterPosition'].split(',')
                            df['Cell_name'].append(site['Name'])
                            df['Position_Name'].append(file+'_'+site['Name'])
                            df['Original_Filename'].append(file)
                            df['Position_X'].append(coordinates[0])
                            df['Position_Y'].append(coordinates[1])
                            dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                            df['NZslices'].append(dimensions['SizeZ']) 
                            df['NChannel'].append(dimensions['SizeC'])
                            df['Nframes'].append(dimensions['SizeT'])
                            df['N_X'].append(dimensions['SizeX'])
                            df['N_Y'].append(dimensions['SizeY'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                            df['AcquisitionDate'].append(date[0])
                            df['AcquisitionTime'].append(date[1])
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                            df['Scale_X'].append(scale['Value'])#Distance Id="X"
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                            df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                            if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                                scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                                df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                            else:
                                df['Scale_Z'].append(1)#Distance Id="Z"

                            #try:
                            if "tilescan" in filename or "TILESCAN" in filename and "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                                tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])

                            else:
                                df['Tilecolumn'].append('1')
                                df['Tilerow'].append('1')   
                            #except Exception as e:
                            #    print(f"{file} error2")




#For single positions

                elif "HardwareSetting" in metadatadict_czi['ImageDocument']['Metadata'].keys() and "S" not in metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys():
                    
                    new_list=metadatadict_czi['ImageDocument']['Metadata']['HardwareSetting']['ParameterCollection']
                    for i in new_list: #i is every dict in the list 
#                         print(i)
                        if 'MTBStageAxisX' in i.values():
                            df['Cell_name'].append(file)
                            df['Position_Name'].append(file)
                            df['Original_Filename'].append(file)
#                             print(i['Position']['value'],i['Id'])
                            df['Position_X'].append(i['Position']['value'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                        if 'MTBStageAxisY' in i.values():
                            df['Position_Y'].append(i['Position']['value'])
                            dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                            df['NZslices'].append(dimensions['SizeZ']) 
                            df['NChannel'].append(dimensions['SizeC'])
                            df['Nframes'].append(dimensions['SizeT'])
                            df['N_X'].append(dimensions['SizeX'])
                            df['N_Y'].append(dimensions['SizeY'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                            df['AcquisitionDate'].append(date[0])
                            df['AcquisitionTime'].append(date[1])
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                            df['Scale_X'].append(scale['Value'])#Distance Id="X"
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                            df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                            if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                                scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                                df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                            else:
                                df['Scale_Z'].append(1)#Distance Id="Z"
                            #try:
                            if "tilescan" in filename or "TILESCAN" in filename and "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                                tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])

                            else:
                                df['Tilecolumn'].append('1')
                                df['Tilerow'].append('1')   
                        # except Exception as e:
                            #   print(f"{file} error3")

                    
                else:
                    print(file)

            
                

for key,value in df.items():
    print(key,len(df[key]))
pddf4=pd.DataFrame(df)
#pddf=pddf.drop_duplicates(subset=['Position_Name'])
#pddf
#pddf.to_csv('/Users/secchim/Documents/VWF_067_MS220909_metadata3.csv', index=False) #only works for dataframes #otherwise you get row numbers

#pddf=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in df.items() ]))#fills empty with na
pddf4



VWF_053_MS220208_homeostasis_firstheadpiece (GrabImage).czi
VWF_053_MS220208_homeostasis_firstheadpiece (GrabImage)noname.czi
Position_Name 186
Cell_name 186
Position_X 186
Position_Y 186
AcquisitionDate 186
AcquisitionTime 186
NChannel 186
Nframes 186
NZslices 186
Tilerow 186
Tilecolumn 186
N_X 186
N_Y 186
Original_Filename 186
Scale_X 186
Scale_Y 186
Scale_Z 186


,Position_Name,Cell_name,Position_X,Position_Y,AcquisitionDate,AcquisitionTime,NChannel,Nframes,NZslices,Tilerow,Tilecolumn,N_X,N_Y,Original_Filename,Scale_X,Scale_Y,Scale_Z
0,Snap-2212.czi,Snap-2212.czi,8030.9,-16007.6,2022-02-08,12:39:55.1478358Z,4,1,1,1,1,1168,1168,Snap-2212.czi,2.136709e-07,2.136709e-07,1.000000
1,Snap-2213.czi,Snap-2213.czi,9225.7,-14326.8,2022-02-08,12:52:50.0272013Z,4,1,1,1,1,1168,1168,Snap-2213.czi,3.632406e-07,3.632406e-07,1.000000
2,Snap-2215.czi,Snap-2215.czi,10196.4,-14908.6,2022-02-08,13:16:42.7814303Z,4,1,1,1,1,1168,1168,Snap-2215.czi,3.632406e-07,3.632406e-07,1.000000
3,VWF_053_MS220208_M3_P3-LSM Plus Processing-01.czi,VWF_053_MS220208_M3_P3-LSM Plus Processing-01.czi,7834.8,-16245.999,2022-02-08,10:07:02.7162456Z,3,1,1,1,1,1168,1168,VWF_053_MS220208_M3_P3-LSM Plus Processing-01.czi,2.136709e-07,2.136709e-07,1.000000
4,VWF_053_MS220208_M3_P4-LSM Plus Processing-02.czi,VWF_053_MS220208_M3_P4-LSM Plus Processing-02.czi,8375.5,-16177.1,2022-02-08,10:10:38.2744187Z,3,1,1,1,1,1168,1168,VWF_053_MS220208_M3_P4-LSM Plus Processing-02.czi,2.136709e-07,2.136709e-07,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,VWF_053_MS220209_M1_movie7depletion.czi_P12kuo,P12kuo,9683.8,-14208.6,2022-02-09,17:48:03.0164471Z,4,1,10,1,1,6592,4565,VWF_053_MS220209_M1_movie7depletion.czi,4.874368e-07,4.874368e-07,0.000003
182,VWF_053_MS220209_M1_movie7depletion.czi_P16DP,P16DP,10563.8,-13951.6,2022-02-09,17:48:03.0164471Z,4,1,10,1,1,6592,4565,VWF_053_MS220209_M1_movie7depletion.czi,4.874368e-07,4.874368e-07,0.000003
183,VWF_053_MS220209_M1_movie7depletion.czi_P26kuo...,P26kuovascu,11031.7,-13125.4,2022-02-09,17:48:03.0164471Z,4,1,10,1,1,6592,4565,VWF_053_MS220209_M1_movie7depletion.czi,4.874368e-07,4.874368e-07,0.000003
184,VWF_053_MS220209_M1_P9zstack.czi_P9,P9,8937.5,-14124.6,2022-02-09,13:32:11.9998909Z,3,1,5,1,1,1168,1168,VWF_053_MS220209_M1_P9zstack.czi,3.632406e-07,3.632406e-07,0.000003


In [9]:

import os
import re
import json
import pandas as pd
import numpy as np
import czifile as zis

#empty dictionary
df={
    'Position_Name':[],
    'Cell_name':[],
    'Position_X':[],
    'Position_Y':[],
    'AcquisitionDate':[],
    'AcquisitionTime':[],
    'NChannel':[],
    'Nframes':[],
    'NZslices':[],
    'Tilerow':[],
    'Tilecolumn':[],
    'N_X':[],
    'N_Y':[],
    'Original_Filename':[],
    'Scale_X':[],
    'Scale_Y':[],
    'Scale_Z':[]
}


filepath='/Volumes/LoCelso/Secchi_Marine'

folder_list=['VWF_060_MS220406_IVMpltdepletion24h']
remove_list=[]


for folder in folder_list:
    filepath=os.path.join(filepath,folder)

    for root, dirs, files in os.walk(filepath): # will open up all the folders, dirs is all the name of the folder it finds, files will contain all the filenames it finds
        for file in files:
            if file.endswith('.czi'):#need to be iterating through file
                duplicates=re.findall(".*(?=-LSM)",file)
                if len(duplicates)>1:
                    remove_list.append(duplicates[0]+'.czi') 
#                    print("test", file,duplicates[0])

        
        for file in files: 
            if file not in remove_list and file.endswith('.czi'):
                #print(file)
                filename=os.path.join(root,file)#looks at the directory used, joins the strings
                try:
                    czi = zis.CziFile(filename)
                except Exception as e:
                    print(f"{file} error {e}")
                    #print(filename)
                    # parse the XML into a dictionary
                metadatadict_czi = czi.metadata(raw=False)


                    

# For multiple positions per file wich should include movies and tilescans
                if "S" in metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys():
                    allscenes = metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['S']['Scenes']['Scene']
#                     if file == 'VWF_043_MS211125_m4-softwareautofocus_gfpref_1.czi':
#                         print(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image'].items())
                    if isinstance(allscenes, dict):
                        coordinates=allscenes['CenterPosition'].split(',')
                        df['Cell_name'].append(allscenes['Name'])
                        df['Position_Name'].append(file+'_'+allscenes['Name'])#changed from site to allscenes
                        df['Original_Filename'].append(file)
                        df['Position_X'].append(coordinates[0])
                        df['Position_Y'].append(coordinates[1])
                        dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                        df['NZslices'].append(dimensions['SizeZ']) 
                        df['NChannel'].append(dimensions['SizeC'])
                        df['Nframes'].append(dimensions['SizeT'])
                        df['N_X'].append(dimensions['SizeX'])
                        df['N_Y'].append(dimensions['SizeY'])
                        date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                        df['AcquisitionDate'].append(date[0])
                        df['AcquisitionTime'].append(date[1])
                        scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                        df['Scale_X'].append(scale['Value'])#Distance Id="X"
                        scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                        df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                        if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                            df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                        else:
                            df['Scale_Z'].append(1)#Distance Id="Z"

                        # try:
                        if "tilescan" in filename.casefold() and "RegionsSetup" in metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups'].keys(): 
                        #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            
                            tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                            if isinstance(tile, list):
                                for i in tile: #going inside the list !!!!
                                    #print(i)
                                    if i['IsUsedForAcquisition']==True:
                                        df['Tilecolumn'].append(i['Columns'])
                                        df['Tilerow'].append(i['Rows'])   
                                    else: 
                                        print(f'{file}IsUsedForAcquisition Not true')                                    

                            #will not work for single positions so the else is required
                            else:
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])
                
                        else:
                            df['Tilecolumn'].append('1')
                            df['Tilerow'].append('1')   
                        # except Exception as e:
                            # print(f"{file} error1 {e}")

                    else:
                            

                        for site in allscenes:#when scenes is a list you have to iterate through each element
                            #print(allscenes,file)
                            coordinates=site['CenterPosition'].split(',')
                            df['Cell_name'].append(site['Name'])
                            df['Position_Name'].append(file+'_'+site['Name'])
                            df['Original_Filename'].append(file)
                            df['Position_X'].append(coordinates[0])
                            df['Position_Y'].append(coordinates[1])
                            dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                            df['NZslices'].append(dimensions['SizeZ']) 
                            df['NChannel'].append(dimensions['SizeC'])
                            df['Nframes'].append(dimensions['SizeT'])
                            df['N_X'].append(dimensions['SizeX'])
                            df['N_Y'].append(dimensions['SizeY'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                            df['AcquisitionDate'].append(date[0])
                            df['AcquisitionTime'].append(date[1])
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                            df['Scale_X'].append(scale['Value'])#Distance Id="X"
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                            df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                            if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                                scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                                df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                            else:
                                df['Scale_Z'].append(1)#Distance Id="Z"

                            #try:
                            if "tilescan" in filename or "TILESCAN" in filename and "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                                tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])

                            else:
                                df['Tilecolumn'].append('1')
                                df['Tilerow'].append('1')   
                            #except Exception as e:
                            #    print(f"{file} error2")




#For single positions

                elif "HardwareSetting" in metadatadict_czi['ImageDocument']['Metadata'].keys() and "S" not in metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys():
                    
                    new_list=metadatadict_czi['ImageDocument']['Metadata']['HardwareSetting']['ParameterCollection']
                    for i in new_list: #i is every dict in the list 
#                         print(i)
                        if 'MTBStageAxisX' in i.values():
                            df['Cell_name'].append(file)
                            df['Position_Name'].append(file)
                            df['Original_Filename'].append(file)
#                             print(i['Position']['value'],i['Id'])
                            df['Position_X'].append(i['Position']['value'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                        if 'MTBStageAxisY' in i.values():
                            df['Position_Y'].append(i['Position']['value'])
                            dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                            df['NZslices'].append(dimensions['SizeZ']) 
                            df['NChannel'].append(dimensions['SizeC'])
                            df['Nframes'].append(dimensions['SizeT'])
                            df['N_X'].append(dimensions['SizeX'])
                            df['N_Y'].append(dimensions['SizeY'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                            df['AcquisitionDate'].append(date[0])
                            df['AcquisitionTime'].append(date[1])
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                            df['Scale_X'].append(scale['Value'])#Distance Id="X"
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                            df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                            if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                                scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                                df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                            else:
                                df['Scale_Z'].append(1)#Distance Id="Z"
                            #try:
                            if "tilescan" in filename or "TILESCAN" in filename and "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                                tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])

                            else:
                                df['Tilecolumn'].append('1')
                                df['Tilerow'].append('1')   
                        # except Exception as e:
                            #   print(f"{file} error3")

                    
                else:
                    print(file)

            
                

for key,value in df.items():
    print(key,len(df[key]))
pddf5=pd.DataFrame(df)
#pddf=pddf.drop_duplicates(subset=['Position_Name'])
#pddf
#pddf.to_csv('/Users/secchim/Documents/VWF_067_MS220909_metadata3.csv', index=False) #only works for dataframes #otherwise you get row numbers

#pddf=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in df.items() ]))#fills empty with na
pddf5



Position_Name 54
Cell_name 54
Position_X 54
Position_Y 54
AcquisitionDate 54
AcquisitionTime 54
NChannel 54
Nframes 54
NZslices 54
Tilerow 54
Tilecolumn 54
N_X 54
N_Y 54
Original_Filename 54
Scale_X 54
Scale_Y 54
Scale_Z 54


,Position_Name,Cell_name,Position_X,Position_Y,AcquisitionDate,AcquisitionTime,NChannel,Nframes,NZslices,Tilerow,Tilecolumn,N_X,N_Y,Original_Filename,Scale_X,Scale_Y,Scale_Z
0,VWF_060_MS220408_M1Snap-2421.czi,VWF_060_MS220408_M1Snap-2421.czi,-14726.6,-13937.5,2022-04-08,08:13:01.5831292Z,4,1,1,1,1,512,512,VWF_060_MS220408_M1Snap-2421.czi,8.286426e-07,8.286426e-07,1.000000
1,VWF_060_MS220408_M1_SNAP1.czi,VWF_060_MS220408_M1_SNAP1.czi,-13127.8,-12994.6,2022-04-08,10:17:47.1077361Z,4,1,1,1,1,512,512,VWF_060_MS220408_M1_SNAP1.czi,4.874368e-07,4.874368e-07,1.000000
2,VWF_060_MS220408_M1_SNAP2.czi,VWF_060_MS220408_M1_SNAP2.czi,-12880.5,-13154.4,2022-04-08,10:19:36.5139566Z,4,1,1,1,1,512,512,VWF_060_MS220408_M1_SNAP2.czi,8.286426e-07,8.286426e-07,1.000000
3,VWF_060_MS220408_M1_SNAP3.czi,VWF_060_MS220408_M1_SNAP3.czi,-12916.5,-13284.1,2022-04-08,10:21:38.7086044Z,4,1,1,1,1,512,512,VWF_060_MS220408_M1_SNAP3.czi,4.874368e-07,4.874368e-07,1.000000
4,VWF_060_MS220408_M1_SNAP_p4.czi,VWF_060_MS220408_M1_SNAP_p4.czi,-12279.2,-12878,2022-04-08,10:40:22.126823Z,4,1,1,1,1,512,512,VWF_060_MS220408_M1_SNAP_p4.czi,8.286426e-07,8.286426e-07,1.000000
5,VWF_060_MS220408_M1_MOVIECHECKSTAGE.czi,VWF_060_MS220408_M1_MOVIECHECKSTAGE.czi,-14832.8,-12829.1,2022-04-08,10:43:57.5609172Z,4,1,1,1,1,512,512,VWF_060_MS220408_M1_MOVIECHECKSTAGE.czi,8.286426e-07,8.286426e-07,1.000000
6,VWF_060_MS220408_M1_P1DP-LSM Plus Processing-0...,VWF_060_MS220408_M1_P1DP-LSM Plus Processing-0...,-14475.9,-13710.5,2022-04-08,08:42:20.8322401Z,3,1,1,1,1,1168,1168,VWF_060_MS220408_M1_P1DP-LSM Plus Processing-0...,2.136709e-07,2.136709e-07,1.000000
7,VWF_060_MS220408_M1_P2-LSM Plus Processing-02.czi,VWF_060_MS220408_M1_P2-LSM Plus Processing-02.czi,-14478.9,-13900.4,2022-04-08,08:49:25.3336491Z,4,1,1,1,1,1168,1168,VWF_060_MS220408_M1_P2-LSM Plus Processing-02.czi,2.136709e-07,2.136709e-07,1.000000
8,VWF_060_MS220408_M1_P3-LSM Plus Processing-03.czi,VWF_060_MS220408_M1_P3-LSM Plus Processing-03.czi,-12639.3,-14241.8,2022-04-08,08:51:06.4001624Z,4,1,1,1,1,1168,1168,VWF_060_MS220408_M1_P3-LSM Plus Processing-03.czi,2.136709e-07,2.136709e-07,1.000000
9,VWF_060_MS220408_M1_P4-LSM Plus Processing-04.czi,VWF_060_MS220408_M1_P4-LSM Plus Processing-04.czi,-11958.2,-13984.1,2022-04-08,08:56:17.0954529Z,4,1,1,1,1,1168,1168,VWF_060_MS220408_M1_P4-LSM Plus Processing-04.czi,2.136709e-07,2.136709e-07,1.000000


In [10]:

import os
import re
import json
import pandas as pd
import numpy as np
import czifile as zis

#empty dictionary
df={
    'Position_Name':[],
    'Cell_name':[],
    'Position_X':[],
    'Position_Y':[],
    'AcquisitionDate':[],
    'AcquisitionTime':[],
    'NChannel':[],
    'Nframes':[],
    'NZslices':[],
    'Tilerow':[],
    'Tilecolumn':[],
    'N_X':[],
    'N_Y':[],
    'Original_Filename':[],
    'Scale_X':[],
    'Scale_Y':[],
    'Scale_Z':[]
}



filepath='/Volumes/LoCelso/Secchi_Marine'

folder_list=['VWF_062_MS220418_IVMdepletion24h']
remove_list=[]


for folder in folder_list:
    filepath=os.path.join(filepath,folder)

    for root, dirs, files in os.walk(filepath): # will open up all the folders, dirs is all the name of the folder it finds, files will contain all the filenames it finds
        for file in files:
            if file.endswith('.czi'):#need to be iterating through file
                duplicates=re.findall(".*(?=-LSM)",file)
                if len(duplicates)>1:
                    remove_list.append(duplicates[0]+'.czi') 
#                    print("test", file,duplicates[0])

        
        for file in files: 
            if file not in remove_list and file.endswith('.czi'):
                #print(file)
                filename=os.path.join(root,file)#looks at the directory used, joins the strings
                try:
                    czi = zis.CziFile(filename)
                except Exception as e:
                    print(f"{file} error {e}")
                    #print(filename)
                    # parse the XML into a dictionary
                metadatadict_czi = czi.metadata(raw=False)


                    

# For multiple positions per file wich should include movies and tilescans
                if "S" in metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys():
                    allscenes = metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['S']['Scenes']['Scene']
#                     if file == 'VWF_043_MS211125_m4-softwareautofocus_gfpref_1.czi':
#                         print(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image'].items())
                    if isinstance(allscenes, dict):
                        coordinates=allscenes['CenterPosition'].split(',')
                        df['Cell_name'].append(allscenes['Name'])
                        df['Position_Name'].append(file+'_'+allscenes['Name'])#changed from site to allscenes
                        df['Original_Filename'].append(file)
                        df['Position_X'].append(coordinates[0])
                        df['Position_Y'].append(coordinates[1])
                        dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                        df['NZslices'].append(dimensions['SizeZ']) 
                        df['NChannel'].append(dimensions['SizeC'])
                        df['Nframes'].append(dimensions['SizeT'])
                        df['N_X'].append(dimensions['SizeX'])
                        df['N_Y'].append(dimensions['SizeY'])
                        date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                        df['AcquisitionDate'].append(date[0])
                        df['AcquisitionTime'].append(date[1])
                        scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                        df['Scale_X'].append(scale['Value'])#Distance Id="X"
                        scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                        df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                        if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                            df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                        else:
                            df['Scale_Z'].append(1)#Distance Id="Z"

                        # try:
                        if "tilescan" in filename.casefold() and "RegionsSetup" in metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups'].keys(): 
                        #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            
                            tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                            if isinstance(tile, list):
                                for i in tile: #going inside the list !!!!
                                    #print(i)
                                    if i['IsUsedForAcquisition']==True:
                                        df['Tilecolumn'].append(i['Columns'])
                                        df['Tilerow'].append(i['Rows'])   
                                    else: 
                                        print(f'{file}IsUsedForAcquisition Not true')                                    

                            #will not work for single positions so the else is required
                            else:
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])
                
                        else:
                            df['Tilecolumn'].append('1')
                            df['Tilerow'].append('1')   
                        # except Exception as e:
                            # print(f"{file} error1 {e}")

                    else:
                            

                        for site in allscenes:#when scenes is a list you have to iterate through each element
                            #print(allscenes,file)
                            coordinates=site['CenterPosition'].split(',')
                            df['Cell_name'].append(site['Name'])
                            df['Position_Name'].append(file+'_'+site['Name'])
                            df['Original_Filename'].append(file)
                            df['Position_X'].append(coordinates[0])
                            df['Position_Y'].append(coordinates[1])
                            dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                            df['NZslices'].append(dimensions['SizeZ']) 
                            df['NChannel'].append(dimensions['SizeC'])
                            df['Nframes'].append(dimensions['SizeT'])
                            df['N_X'].append(dimensions['SizeX'])
                            df['N_Y'].append(dimensions['SizeY'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                            df['AcquisitionDate'].append(date[0])
                            df['AcquisitionTime'].append(date[1])
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                            df['Scale_X'].append(scale['Value'])#Distance Id="X"
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                            df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                            if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                                scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                                df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                            else:
                                df['Scale_Z'].append(1)#Distance Id="Z"

                            #try:
                            if "tilescan" in filename or "TILESCAN" in filename and "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                                tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])

                            else:
                                df['Tilecolumn'].append('1')
                                df['Tilerow'].append('1')   
                            #except Exception as e:
                            #    print(f"{file} error2")




#For single positions

                elif "HardwareSetting" in metadatadict_czi['ImageDocument']['Metadata'].keys() and "S" not in metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys():
                    
                    new_list=metadatadict_czi['ImageDocument']['Metadata']['HardwareSetting']['ParameterCollection']
                    for i in new_list: #i is every dict in the list 
#                         print(i)
                        if 'MTBStageAxisX' in i.values():
                            df['Cell_name'].append(file)
                            df['Position_Name'].append(file)
                            df['Original_Filename'].append(file)
#                             print(i['Position']['value'],i['Id'])
                            df['Position_X'].append(i['Position']['value'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                        if 'MTBStageAxisY' in i.values():
                            df['Position_Y'].append(i['Position']['value'])
                            dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                            df['NZslices'].append(dimensions['SizeZ']) 
                            df['NChannel'].append(dimensions['SizeC'])
                            df['Nframes'].append(dimensions['SizeT'])
                            df['N_X'].append(dimensions['SizeX'])
                            df['N_Y'].append(dimensions['SizeY'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                            df['AcquisitionDate'].append(date[0])
                            df['AcquisitionTime'].append(date[1])
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                            df['Scale_X'].append(scale['Value'])#Distance Id="X"
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                            df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                            if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                                scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                                df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                            else:
                                df['Scale_Z'].append(1)#Distance Id="Z"
                            #try:
                            if "tilescan" in filename or "TILESCAN" in filename and "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                                tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])

                            else:
                                df['Tilecolumn'].append('1')
                                df['Tilerow'].append('1')   
                        # except Exception as e:
                            #   print(f"{file} error3")

                    
                else:
                    print(file)

            
                

for key,value in df.items():
    print(key,len(df[key]))
pddf6=pd.DataFrame(df)
#pddf=pddf.drop_duplicates(subset=['Position_Name'])
#pddf
#pddf.to_csv('/Users/secchim/Documents/VWF_067_MS220909_metadata3.csv', index=False) #only works for dataframes #otherwise you get row numbers

#pddf=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in df.items() ]))#fills empty with na
pddf6



VWF_062_MS220421_M5_TILESCAN_2padded.cziIsUsedForAcquisition Not true
VWF_062_MS220421_M5_TILESCAN_2.cziIsUsedForAcquisition Not true
Position_Name 105
Cell_name 105
Position_X 105
Position_Y 105
AcquisitionDate 105
AcquisitionTime 105
NChannel 105
Nframes 105
NZslices 105
Tilerow 105
Tilecolumn 105
N_X 105
N_Y 105
Original_Filename 105
Scale_X 105
Scale_Y 105
Scale_Z 105


,Position_Name,Cell_name,Position_X,Position_Y,AcquisitionDate,AcquisitionTime,NChannel,Nframes,NZslices,Tilerow,Tilecolumn,N_X,N_Y,Original_Filename,Scale_X,Scale_Y,Scale_Z
0,VWF_062_MS220419_M2_P4-LSM Plus Processing-01.czi,VWF_062_MS220419_M2_P4-LSM Plus Processing-01.czi,5692.5,45188.4,2022-04-19,12:40:15.021941Z,4,1,1,1,1,1168,1168,VWF_062_MS220419_M2_P4-LSM Plus Processing-01.czi,2.136709e-07,2.136709e-07,1.000000
1,VWF_062_MS220419_M2_P5-LSM Plus Processing-01.czi,VWF_062_MS220419_M2_P5-LSM Plus Processing-01.czi,5867.1,45458.8,2022-04-19,12:44:07.4493846Z,4,1,1,1,1,1168,1168,VWF_062_MS220419_M2_P5-LSM Plus Processing-01.czi,2.136709e-07,2.136709e-07,1.000000
2,VWF_062_MS220419_M2_P5_2-LSM Plus Processing-0...,VWF_062_MS220419_M2_P5_2-LSM Plus Processing-0...,5866.5,45458.8,2022-04-19,13:29:13.4278429Z,4,1,1,1,1,1168,1168,VWF_062_MS220419_M2_P5_2-LSM Plus Processing-0...,2.136709e-07,2.136709e-07,1.000000
3,VWF_062_MS220419_M2_P8-LSM Plus Processing-03.czi,VWF_062_MS220419_M2_P8-LSM Plus Processing-03.czi,6264,43738.5,2022-04-19,12:52:20.4159873Z,4,1,1,1,1,1168,1168,VWF_062_MS220419_M2_P8-LSM Plus Processing-03.czi,2.136709e-07,2.136709e-07,1.000000
4,VWF_062_MS220419_M2_P9-LSM Plus Processing-04.czi,VWF_062_MS220419_M2_P9-LSM Plus Processing-04.czi,6354.9,43642.5,2022-04-19,12:54:04.4940672Z,4,1,1,1,1,1168,1168,VWF_062_MS220419_M2_P9-LSM Plus Processing-04.czi,2.136709e-07,2.136709e-07,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,VWF_062_MS220421_M5_MOVIE1.czi_P17mk,P17mk,-2047.2,45841.5,2022-04-21,10:45:31.7330974Z,4,1,8,1,1,4580,2410,VWF_062_MS220421_M5_MOVIE1.czi,4.874368e-07,4.874368e-07,0.000003
101,VWF_062_MS220421_M4_zstacks2.czi_P2,P2,-2005.7,43421.3,2022-04-21,15:08:25.1653604Z,4,1,8,1,1,1176,2252,VWF_062_MS220421_M4_zstacks2.czi,8.286426e-07,8.286426e-07,0.000003
102,VWF_062_MS220421_M4_zstacks2.czi_P3,P3,-1455.2,44862.5,2022-04-21,15:08:25.1653604Z,4,1,8,1,1,1176,2252,VWF_062_MS220421_M4_zstacks2.czi,8.286426e-07,8.286426e-07,0.000003
103,VWF_062_MS220421_M5_TILESCAN_2.czi_TR2,TR2,-1196.895,44587.71,2022-04-21,10:34:14.3526844Z,4,1,10,2,5,2358,973,VWF_062_MS220421_M5_TILESCAN_2.czi,8.286426e-07,8.286426e-07,0.000003


In [11]:

import os
import re
import json
import pandas as pd
import numpy as np
import czifile as zis

#empty dictionary
df={
    'Position_Name':[],
    'Cell_name':[],
    'Position_X':[],
    'Position_Y':[],
    'AcquisitionDate':[],
    'AcquisitionTime':[],
    'NChannel':[],
    'Nframes':[],
    'NZslices':[],
    'Tilerow':[],
    'Tilecolumn':[],
    'N_X':[],
    'N_Y':[],
    'Original_Filename':[],
    'Scale_X':[],
    'Scale_Y':[],
    'Scale_Z':[]
}



filepath='/Volumes/LoCelso/Secchi_Marine'

folder_list=['VWF_063_MS220428_IVMdepletion24h']
remove_list=[]


for folder in folder_list:
    filepath=os.path.join(filepath,folder)

    for root, dirs, files in os.walk(filepath): # will open up all the folders, dirs is all the name of the folder it finds, files will contain all the filenames it finds
        for file in files:
            if file.endswith('.czi'):#need to be iterating through file
                duplicates=re.findall(".*(?=-LSM)",file)
                if len(duplicates)>1:
                    remove_list.append(duplicates[0]+'.czi') 
#                    print("test", file,duplicates[0])

        
        for file in files: 
            if file not in remove_list and file.endswith('.czi'):
                #print(file)
                filename=os.path.join(root,file)#looks at the directory used, joins the strings
                try:
                    czi = zis.CziFile(filename)
                except Exception as e:
                    print(f"{file} error {e}")
                    #print(filename)
                    # parse the XML into a dictionary
                metadatadict_czi = czi.metadata(raw=False)


                    

# For multiple positions per file wich should include movies and tilescans
                if "S" in metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys():
                    allscenes = metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['S']['Scenes']['Scene']
#                     if file == 'VWF_043_MS211125_m4-softwareautofocus_gfpref_1.czi':
#                         print(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image'].items())
                    if isinstance(allscenes, dict):
                        coordinates=allscenes['CenterPosition'].split(',')
                        df['Cell_name'].append(allscenes['Name'])
                        df['Position_Name'].append(file+'_'+allscenes['Name'])#changed from site to allscenes
                        df['Original_Filename'].append(file)
                        df['Position_X'].append(coordinates[0])
                        df['Position_Y'].append(coordinates[1])
                        dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                        df['NZslices'].append(dimensions['SizeZ']) 
                        df['NChannel'].append(dimensions['SizeC'])
                        df['Nframes'].append(dimensions['SizeT'])
                        df['N_X'].append(dimensions['SizeX'])
                        df['N_Y'].append(dimensions['SizeY'])
                        date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                        df['AcquisitionDate'].append(date[0])
                        df['AcquisitionTime'].append(date[1])
                        scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                        df['Scale_X'].append(scale['Value'])#Distance Id="X"
                        scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                        df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                        if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                            df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                        else:
                            df['Scale_Z'].append(1)#Distance Id="Z"

                        # try:
                        if "tilescan" in filename.casefold() and "RegionsSetup" in metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups'].keys(): 
                        #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            
                            tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                            if isinstance(tile, list):
                                for i in tile: #going inside the list !!!!
                                    #print(i)
                                    if i['IsUsedForAcquisition']==True:
                                        df['Tilecolumn'].append(i['Columns'])
                                        df['Tilerow'].append(i['Rows'])   
                                    else: 
                                        print(f'{file}IsUsedForAcquisition Not true')                                    

                            #will not work for single positions so the else is required
                            else:
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])
                
                        else:
                            df['Tilecolumn'].append('1')
                            df['Tilerow'].append('1')   
                        # except Exception as e:
                            # print(f"{file} error1 {e}")

                    else:
                            

                        for site in allscenes:#when scenes is a list you have to iterate through each element
                            #print(allscenes,file)
                            coordinates=site['CenterPosition'].split(',')
                            df['Cell_name'].append(site['Name'])
                            df['Position_Name'].append(file+'_'+site['Name'])
                            df['Original_Filename'].append(file)
                            df['Position_X'].append(coordinates[0])
                            df['Position_Y'].append(coordinates[1])
                            dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                            df['NZslices'].append(dimensions['SizeZ']) 
                            df['NChannel'].append(dimensions['SizeC'])
                            df['Nframes'].append(dimensions['SizeT'])
                            df['N_X'].append(dimensions['SizeX'])
                            df['N_Y'].append(dimensions['SizeY'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                            df['AcquisitionDate'].append(date[0])
                            df['AcquisitionTime'].append(date[1])
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                            df['Scale_X'].append(scale['Value'])#Distance Id="X"
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                            df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                            if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                                scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                                df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                            else:
                                df['Scale_Z'].append(1)#Distance Id="Z"

                            #try:
                            if "tilescan" in filename or "TILESCAN" in filename and "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                                tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])

                            else:
                                df['Tilecolumn'].append('1')
                                df['Tilerow'].append('1')   
                            #except Exception as e:
                            #    print(f"{file} error2")




#For single positions

                elif "HardwareSetting" in metadatadict_czi['ImageDocument']['Metadata'].keys() and "S" not in metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys():
                    
                    new_list=metadatadict_czi['ImageDocument']['Metadata']['HardwareSetting']['ParameterCollection']
                    for i in new_list: #i is every dict in the list 
#                         print(i)
                        if 'MTBStageAxisX' in i.values():
                            df['Cell_name'].append(file)
                            df['Position_Name'].append(file)
                            df['Original_Filename'].append(file)
#                             print(i['Position']['value'],i['Id'])
                            df['Position_X'].append(i['Position']['value'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                        if 'MTBStageAxisY' in i.values():
                            df['Position_Y'].append(i['Position']['value'])
                            dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                            df['NZslices'].append(dimensions['SizeZ']) 
                            df['NChannel'].append(dimensions['SizeC'])
                            df['Nframes'].append(dimensions['SizeT'])
                            df['N_X'].append(dimensions['SizeX'])
                            df['N_Y'].append(dimensions['SizeY'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                            df['AcquisitionDate'].append(date[0])
                            df['AcquisitionTime'].append(date[1])
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                            df['Scale_X'].append(scale['Value'])#Distance Id="X"
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                            df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                            if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                                scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                                df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                            else:
                                df['Scale_Z'].append(1)#Distance Id="Z"
                            #try:
                            if "tilescan" in filename or "TILESCAN" in filename and "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                                tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])

                            else:
                                df['Tilecolumn'].append('1')
                                df['Tilerow'].append('1')   
                        # except Exception as e:
                            #   print(f"{file} error3")

                    
                else:
                    print(file)

            
                

for key,value in df.items():
    print(key,len(df[key]))
pddf7=pd.DataFrame(df)
#pddf=pddf.drop_duplicates(subset=['Position_Name'])
#pddf
#pddf.to_csv('/Users/secchim/Documents/VWF_067_MS220909_metadata3.csv', index=False) #only works for dataframes #otherwise you get row numbers

#pddf=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in df.items() ]))#fills empty with na
pddf7



VWF_063_MS220428_m2_tilescan.cziIsUsedForAcquisition Not true
VWF_063_MS220428_m2_tilescan2.cziIsUsedForAcquisition Not true
VWF_063_MS220428_m2_tilescanpadded-stitched.cziIsUsedForAcquisition Not true
VWF_063_MS220428_m2_tilescan2padded-stitched.cziIsUsedForAcquisition Not true
VWF_063_MS220428_m2_tilescan.cziIsUsedForAcquisition Not true
VWF_063_MS220428_m2_tilescan2.cziIsUsedForAcquisition Not true
VWF_063_MS220428_m2_tilescan2padded.cziIsUsedForAcquisition Not true
VWF_063_MS220428_m2_tilescanpadded.cziIsUsedForAcquisition Not true
Position_Name 98
Cell_name 98
Position_X 98
Position_Y 98
AcquisitionDate 98
AcquisitionTime 98
NChannel 98
Nframes 98
NZslices 98
Tilerow 98
Tilecolumn 98
N_X 98
N_Y 98
Original_Filename 98
Scale_X 98
Scale_Y 98
Scale_Z 98


,Position_Name,Cell_name,Position_X,Position_Y,AcquisitionDate,AcquisitionTime,NChannel,Nframes,NZslices,Tilerow,Tilecolumn,N_X,N_Y,Original_Filename,Scale_X,Scale_Y,Scale_Z
0,VWF_063_MS220428_m1_p2.czi,VWF_063_MS220428_m1_p2.czi,25288.6,-13137.1,2022-04-28,08:09:28.234024Z,3,1,1,1,1,1168,1168,VWF_063_MS220428_m1_p2.czi,2.136709e-07,2.136709e-07,1.000000
1,VWF_063_MS220428_m1_p3.czi,VWF_063_MS220428_m1_p3.czi,25288.4,-13137.2,2022-04-28,08:11:33.3708833Z,4,1,1,1,1,1168,1168,VWF_063_MS220428_m1_p3.czi,2.136709e-07,2.136709e-07,1.000000
2,VWF_063_MS220428_m1_p4.czi,VWF_063_MS220428_m1_p4.czi,25651,-13178.5,2022-04-28,08:13:12.1112668Z,4,1,1,1,1,1168,1168,VWF_063_MS220428_m1_p4.czi,2.136709e-07,2.136709e-07,1.000000
3,VWF_063_MS220428_m1_p5.czi,VWF_063_MS220428_m1_p5.czi,25534.3,-13094.5,2022-04-28,08:14:20.6133023Z,4,1,1,1,1,1168,1168,VWF_063_MS220428_m1_p5.czi,2.136709e-07,2.136709e-07,1.000000
4,VWF_063_MS220428_m1_p6.czi,VWF_063_MS220428_m1_p6.czi,25989.4,-13164.6,2022-04-28,08:17:32.4127104Z,4,1,1,1,1,1168,1168,VWF_063_MS220428_m1_p6.czi,2.136709e-07,2.136709e-07,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,VWF_063_MS220428_m1_tilescanpadded.czi_TR1,TR1,24891.489,-13231.197,2022-04-28,09:11:29.0780392Z,4,1,50,2,7,3584,1024,VWF_063_MS220428_m1_tilescanpadded.czi,8.286426e-07,8.286426e-07,0.000003
94,VWF_063_MS220428_m2_tilescan.czi_TR1,TR1,25519.929,-12906.042,2022-04-28,13:35:57.8956303Z,4,1,12,2,10,4659,973,VWF_063_MS220428_m2_tilescan.czi,8.286426e-07,8.286426e-07,0.000003
95,VWF_063_MS220428_m2_tilescan2.czi_TR2,TR2,25611.637,-11723.671,2022-04-28,14:02:57.8304059Z,4,1,12,2,11,5120,974,VWF_063_MS220428_m2_tilescan2.czi,8.286426e-07,8.286426e-07,0.000003
96,VWF_063_MS220428_m2_tilescan2padded.czi_TR2,TR2,25611.637,-11723.671,2022-04-28,14:02:57.8304059Z,4,1,70,2,11,5632,1024,VWF_063_MS220428_m2_tilescan2padded.czi,8.286426e-07,8.286426e-07,0.000003


In [12]:

import os
import re
import json
import pandas as pd
import numpy as np
import czifile as zis

#empty dictionary
df={
    'Position_Name':[],
    'Cell_name':[],
    'Position_X':[],
    'Position_Y':[],
    'AcquisitionDate':[],
    'AcquisitionTime':[],
    'NChannel':[],
    'Nframes':[],
    'NZslices':[],
    'Tilerow':[],
    'Tilecolumn':[],
    'N_X':[],
    'N_Y':[],
    'Original_Filename':[],
    'Scale_X':[],
    'Scale_Y':[],
    'Scale_Z':[]
}



filepath='/Volumes/LoCelso/Secchi_Marine'

folder_list=['VWF_066_MS220519_IVMdepletion24h']
#VWF_062_MS220418_IVMdepletion24h
remove_list=[]


for folder in folder_list:
    filepath=os.path.join(filepath,folder)

    for root, dirs, files in os.walk(filepath): # will open up all the folders, dirs is all the name of the folder it finds, files will contain all the filenames it finds
        for file in files:
            if file.endswith('.czi'):#need to be iterating through file
                duplicates=re.findall(".*(?=-LSM)",file)
                if len(duplicates)>1:
                    remove_list.append(duplicates[0]+'.czi') 
#                    print("test", file,duplicates[0])

        
        for file in files: 
            if file not in remove_list and file.endswith('.czi'):
                #print(file)
                filename=os.path.join(root,file)#looks at the directory used, joins the strings
                try:
                    czi = zis.CziFile(filename)
                except Exception as e:
                    print(f"{file} error {e}")
                    #print(filename)
                    # parse the XML into a dictionary
                metadatadict_czi = czi.metadata(raw=False)


                    

# For multiple positions per file wich should include movies and tilescans
                if "S" in metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys():
                    allscenes = metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['S']['Scenes']['Scene']
#                     if file == 'VWF_043_MS211125_m4-softwareautofocus_gfpref_1.czi':
#                         print(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image'].items())
                    if isinstance(allscenes, dict):
                        coordinates=allscenes['CenterPosition'].split(',')
                        df['Cell_name'].append(allscenes['Name'])
                        df['Position_Name'].append(file+'_'+allscenes['Name'])#changed from site to allscenes
                        df['Original_Filename'].append(file)
                        df['Position_X'].append(coordinates[0])
                        df['Position_Y'].append(coordinates[1])
                        dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                        df['NZslices'].append(dimensions['SizeZ']) 
                        df['NChannel'].append(dimensions['SizeC'])
                        df['Nframes'].append(dimensions['SizeT'])
                        df['N_X'].append(dimensions['SizeX'])
                        df['N_Y'].append(dimensions['SizeY'])
                        date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                        df['AcquisitionDate'].append(date[0])
                        df['AcquisitionTime'].append(date[1])
                        scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                        df['Scale_X'].append(scale['Value'])#Distance Id="X"
                        scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                        df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                        if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                            df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                        else:
                            df['Scale_Z'].append(1)#Distance Id="Z"

                        # try:
                        if "tilescan" in filename.casefold() and "RegionsSetup" in metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups'].keys(): 
                        #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            
                            tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                            if isinstance(tile, list):
                                for i in tile: #going inside the list !!!!
                                    #print(i)
                                    if i['IsUsedForAcquisition']==True:
                                        df['Tilecolumn'].append(i['Columns'])
                                        df['Tilerow'].append(i['Rows'])   
                                    else: 
                                        print(f'{file}IsUsedForAcquisition Not true')                                    

                            #will not work for single positions so the else is required
                            else:
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])
                
                        else:
                            df['Tilecolumn'].append('1')
                            df['Tilerow'].append('1')   
                        # except Exception as e:
                            # print(f"{file} error1 {e}")

                    else:
                            

                        for site in allscenes:#when scenes is a list you have to iterate through each element
                            #print(allscenes,file)
                            coordinates=site['CenterPosition'].split(',')
                            df['Cell_name'].append(site['Name'])
                            df['Position_Name'].append(file+'_'+site['Name'])
                            df['Original_Filename'].append(file)
                            df['Position_X'].append(coordinates[0])
                            df['Position_Y'].append(coordinates[1])
                            dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                            df['NZslices'].append(dimensions['SizeZ']) 
                            df['NChannel'].append(dimensions['SizeC'])
                            df['Nframes'].append(dimensions['SizeT'])
                            df['N_X'].append(dimensions['SizeX'])
                            df['N_Y'].append(dimensions['SizeY'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                            df['AcquisitionDate'].append(date[0])
                            df['AcquisitionTime'].append(date[1])
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                            df['Scale_X'].append(scale['Value'])#Distance Id="X"
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                            df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                            if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                                scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                                df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                            else:
                                df['Scale_Z'].append(1)#Distance Id="Z"

                            #try:
                            if "tilescan" in filename or "TILESCAN" in filename and "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                                tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])

                            else:
                                df['Tilecolumn'].append('1')
                                df['Tilerow'].append('1')   
                            #except Exception as e:
                            #    print(f"{file} error2")




#For single positions

                elif "HardwareSetting" in metadatadict_czi['ImageDocument']['Metadata'].keys() and "S" not in metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys():
                    
                    new_list=metadatadict_czi['ImageDocument']['Metadata']['HardwareSetting']['ParameterCollection']
                    for i in new_list: #i is every dict in the list 
#                         print(i)
                        if 'MTBStageAxisX' in i.values():
                            df['Cell_name'].append(file)
                            df['Position_Name'].append(file)
                            df['Original_Filename'].append(file)
#                             print(i['Position']['value'],i['Id'])
                            df['Position_X'].append(i['Position']['value'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                        if 'MTBStageAxisY' in i.values():
                            df['Position_Y'].append(i['Position']['value'])
                            dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                            df['NZslices'].append(dimensions['SizeZ']) 
                            df['NChannel'].append(dimensions['SizeC'])
                            df['Nframes'].append(dimensions['SizeT'])
                            df['N_X'].append(dimensions['SizeX'])
                            df['N_Y'].append(dimensions['SizeY'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                            df['AcquisitionDate'].append(date[0])
                            df['AcquisitionTime'].append(date[1])
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                            df['Scale_X'].append(scale['Value'])#Distance Id="X"
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                            df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                            if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                                scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                                df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                            else:
                                df['Scale_Z'].append(1)#Distance Id="Z"
                            #try:
                            if "tilescan" in filename or "TILESCAN" in filename and "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                                tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])

                            else:
                                df['Tilecolumn'].append('1')
                                df['Tilerow'].append('1')   
                        # except Exception as e:
                            #   print(f"{file} error3")

                    
                else:
                    print(file)

            
                

for key,value in df.items():
    print(key,len(df[key]))
pddf8=pd.DataFrame(df)
#pddf=pddf.drop_duplicates(subset=['Position_Name'])
#pddf
#pddf.to_csv('/Users/secchim/Documents/VWF_067_MS220909_metadata3.csv', index=False) #only works for dataframes #otherwise you get row numbers

#pddf=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in df.items() ]))#fills empty with na
pddf8



VWF_066_MS220908_M3 (GrabImage).czi
VWF_066_MS220908_M5 (GrabImage).czi
VWF_066_MS220908_M2 (GrabImage).czi
Position_Name 214
Cell_name 214
Position_X 214
Position_Y 214
AcquisitionDate 214
AcquisitionTime 214
NChannel 214
Nframes 214
NZslices 214
Tilerow 214
Tilecolumn 214
N_X 214
N_Y 214
Original_Filename 214
Scale_X 214
Scale_Y 214
Scale_Z 214


,Position_Name,Cell_name,Position_X,Position_Y,AcquisitionDate,AcquisitionTime,NChannel,Nframes,NZslices,Tilerow,Tilecolumn,N_X,N_Y,Original_Filename,Scale_X,Scale_Y,Scale_Z
0,VWF_066_MS220519_M1_kuovascu-LSM Plus Processi...,VWF_066_MS220519_M1_kuovascu-LSM Plus Processi...,-5049.5,-12778.2,2022-05-19,15:26:32.7000345Z,4,1,1,1,1,1168,1168,VWF_066_MS220519_M1_kuovascu-LSM Plus Processi...,2.136709e-07,2.136709e-07,1.000000
1,VWF_066_MS220519_M1_p4-LSM Plus Processing-02.czi,VWF_066_MS220519_M1_p4-LSM Plus Processing-02.czi,-4785,-14511.1,2022-05-19,14:40:54.637547Z,4,1,1,1,1,1168,1168,VWF_066_MS220519_M1_p4-LSM Plus Processing-02.czi,2.136709e-07,2.136709e-07,1.000000
2,VWF_066_MS220519_M1_p5-LSM Plus Processing-03.czi,VWF_066_MS220519_M1_p5-LSM Plus Processing-03.czi,-4602,-14220.3,2022-05-19,14:39:54.3860866Z,4,1,1,1,1,1168,1168,VWF_066_MS220519_M1_p5-LSM Plus Processing-03.czi,2.136709e-07,2.136709e-07,1.000000
3,VWF_066_MS220519_M1_p6-LSM Plus Processing-04.czi,VWF_066_MS220519_M1_p6-LSM Plus Processing-04.czi,-4714.4,-14060.4,2022-05-19,14:42:10.8766185Z,4,1,1,1,1,1168,1168,VWF_066_MS220519_M1_p6-LSM Plus Processing-04.czi,2.136709e-07,2.136709e-07,1.000000
4,VWF_066_MS220522_M4_p13-LSM Plus Processing-35...,VWF_066_MS220522_M4_p13-LSM Plus Processing-35...,-1095.6,44795.7,2022-05-22,09:08:24.9117674Z,4,1,1,1,1,1168,1168,VWF_066_MS220522_M4_p13-LSM Plus Processing-35...,2.136709e-07,2.136709e-07,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,VWF_066_MS220522_M4_pmovie1.czi_P3kuo,P3kuo,-958.4,43643.3,2022-05-22,09:20:20.2132747Z,4,2,10,1,1,2315,2035,VWF_066_MS220522_M4_pmovie1.czi,4.874368e-07,4.874368e-07,0.000003
210,VWF_066_MS220522_M4_pmovie2.czi_P2kuomk,P2kuomk,-1000.899,42901,2022-05-22,09:26:37.0126733Z,4,21,10,1,1,2316,2036,VWF_066_MS220522_M4_pmovie2.czi,4.874368e-07,4.874368e-07,0.000003
211,VWF_066_MS220522_M4_pmovie2.czi_P5bigDP,P5bigDP,-407.9,43271.7,2022-05-22,09:26:37.0126733Z,4,21,10,1,1,2316,2036,VWF_066_MS220522_M4_pmovie2.czi,4.874368e-07,4.874368e-07,0.000003
212,VWF_066_MS220522_M4_pmovie2.czi_P4_dp,P4_dp,-1286.5,43528.3,2022-05-22,09:26:37.0126733Z,4,21,10,1,1,2316,2036,VWF_066_MS220522_M4_pmovie2.czi,4.874368e-07,4.874368e-07,0.000003


In [13]:

import os
import re
import json
import pandas as pd
import numpy as np
import czifile as zis

#empty dictionary
df={
    'Position_Name':[],
    'Cell_name':[],
    'Position_X':[],
    'Position_Y':[],
    'AcquisitionDate':[],
    'AcquisitionTime':[],
    'NChannel':[],
    'Nframes':[],
    'NZslices':[],
    'Tilerow':[],
    'Tilecolumn':[],
    'N_X':[],
    'N_Y':[],
    'Original_Filename':[],
    'Scale_X':[],
    'Scale_Y':[],
    'Scale_Z':[]
}



filepath='/Volumes/LoCelso/Secchi_Marine'

folder_list=['VWF_067_MS220526_IVMdepletion24h' ]
#VWF_062_MS220418_IVMdepletion24h
remove_list=[]


for folder in folder_list:
    filepath=os.path.join(filepath,folder)

    for root, dirs, files in os.walk(filepath): # will open up all the folders, dirs is all the name of the folder it finds, files will contain all the filenames it finds
        for file in files:
            if file.endswith('.czi'):#need to be iterating through file
                duplicates=re.findall(".*(?=-LSM)",file)
                if len(duplicates)>1:
                    remove_list.append(duplicates[0]+'.czi') 
#                    print("test", file,duplicates[0])

        
        for file in files: 
            if file not in remove_list and file.endswith('.czi'):
                #print(file)
                filename=os.path.join(root,file)#looks at the directory used, joins the strings
                try:
                    czi = zis.CziFile(filename)
                except Exception as e:
                    print(f"{file} error {e}")
                    #print(filename)
                    # parse the XML into a dictionary
                metadatadict_czi = czi.metadata(raw=False)


                    

# For multiple positions per file wich should include movies and tilescans
                if "S" in metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys():
                    allscenes = metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['S']['Scenes']['Scene']
#                     if file == 'VWF_043_MS211125_m4-softwareautofocus_gfpref_1.czi':
#                         print(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image'].items())
                    if isinstance(allscenes, dict):
                        coordinates=allscenes['CenterPosition'].split(',')
                        df['Cell_name'].append(allscenes['Name'])
                        df['Position_Name'].append(file+'_'+allscenes['Name'])#changed from site to allscenes
                        df['Original_Filename'].append(file)
                        df['Position_X'].append(coordinates[0])
                        df['Position_Y'].append(coordinates[1])
                        dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                        df['NZslices'].append(dimensions['SizeZ']) 
                        df['NChannel'].append(dimensions['SizeC'])
                        df['Nframes'].append(dimensions['SizeT'])
                        df['N_X'].append(dimensions['SizeX'])
                        df['N_Y'].append(dimensions['SizeY'])
                        date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                        df['AcquisitionDate'].append(date[0])
                        df['AcquisitionTime'].append(date[1])
                        scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                        df['Scale_X'].append(scale['Value'])#Distance Id="X"
                        scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                        df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                        if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                            df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                        else:
                            df['Scale_Z'].append(1)#Distance Id="Z"

                        # try:
                        if "tilescan" in filename.casefold() and "RegionsSetup" in metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups'].keys(): 
                        #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            
                            tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                            if isinstance(tile, list):
                                for i in tile: #going inside the list !!!!
                                    #print(i)
                                    if i['IsUsedForAcquisition']==True:
                                        df['Tilecolumn'].append(i['Columns'])
                                        df['Tilerow'].append(i['Rows'])   
                                    else: 
                                        print(f'{file}IsUsedForAcquisition Not true')                                    

                            #will not work for single positions so the else is required
                            else:
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])
                
                        else:
                            df['Tilecolumn'].append('1')
                            df['Tilerow'].append('1')   
                        # except Exception as e:
                            # print(f"{file} error1 {e}")

                    else:
                            

                        for site in allscenes:#when scenes is a list you have to iterate through each element
                            #print(allscenes,file)
                            coordinates=site['CenterPosition'].split(',')
                            df['Cell_name'].append(site['Name'])
                            df['Position_Name'].append(file+'_'+site['Name'])
                            df['Original_Filename'].append(file)
                            df['Position_X'].append(coordinates[0])
                            df['Position_Y'].append(coordinates[1])
                            dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                            df['NZslices'].append(dimensions['SizeZ']) 
                            df['NChannel'].append(dimensions['SizeC'])
                            df['Nframes'].append(dimensions['SizeT'])
                            df['N_X'].append(dimensions['SizeX'])
                            df['N_Y'].append(dimensions['SizeY'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                            df['AcquisitionDate'].append(date[0])
                            df['AcquisitionTime'].append(date[1])
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                            df['Scale_X'].append(scale['Value'])#Distance Id="X"
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                            df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                            if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                                scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                                df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                            else:
                                df['Scale_Z'].append(1)#Distance Id="Z"

                            #try:
                            if "tilescan" in filename or "TILESCAN" in filename and "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                                tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])

                            else:
                                df['Tilecolumn'].append('1')
                                df['Tilerow'].append('1')   
                            #except Exception as e:
                            #    print(f"{file} error2")




#For single positions

                elif "HardwareSetting" in metadatadict_czi['ImageDocument']['Metadata'].keys() and "S" not in metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys():
                    
                    new_list=metadatadict_czi['ImageDocument']['Metadata']['HardwareSetting']['ParameterCollection']
                    for i in new_list: #i is every dict in the list 
#                         print(i)
                        if 'MTBStageAxisX' in i.values():
                            df['Cell_name'].append(file)
                            df['Position_Name'].append(file)
                            df['Original_Filename'].append(file)
#                             print(i['Position']['value'],i['Id'])
                            df['Position_X'].append(i['Position']['value'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                        if 'MTBStageAxisY' in i.values():
                            df['Position_Y'].append(i['Position']['value'])
                            dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                            df['NZslices'].append(dimensions['SizeZ']) 
                            df['NChannel'].append(dimensions['SizeC'])
                            df['Nframes'].append(dimensions['SizeT'])
                            df['N_X'].append(dimensions['SizeX'])
                            df['N_Y'].append(dimensions['SizeY'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                            df['AcquisitionDate'].append(date[0])
                            df['AcquisitionTime'].append(date[1])
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                            df['Scale_X'].append(scale['Value'])#Distance Id="X"
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                            df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                            if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                                scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                                df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                            else:
                                df['Scale_Z'].append(1)#Distance Id="Z"
                            #try:
                            if "tilescan" in filename or "TILESCAN" in filename and "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                                tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])

                            else:
                                df['Tilecolumn'].append('1')
                                df['Tilerow'].append('1')   
                        # except Exception as e:
                            #   print(f"{file} error3")

                    
                else:
                    print(file)
            
                

for key,value in df.items():
    print(key,len(df[key]))
pddf9=pd.DataFrame(df)
#pddf=pddf.drop_duplicates(subset=['Position_Name'])
#pddf
#pddf.to_csv('/Users/secchim/Documents/VWF_067_MS220909_metadata3.csv', index=False) #only works for dataframes #otherwise you get row numbers

#pddf=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in df.items() ]))#fills empty with na
pddf9



Position_Name 104
Cell_name 104
Position_X 104
Position_Y 104
AcquisitionDate 104
AcquisitionTime 104
NChannel 104
Nframes 104
NZslices 104
Tilerow 104
Tilecolumn 104
N_X 104
N_Y 104
Original_Filename 104
Scale_X 104
Scale_Y 104
Scale_Z 104


,Position_Name,Cell_name,Position_X,Position_Y,AcquisitionDate,AcquisitionTime,NChannel,Nframes,NZslices,Tilerow,Tilecolumn,N_X,N_Y,Original_Filename,Scale_X,Scale_Y,Scale_Z
0,VWF_067_220526_M2_snap1.czi,VWF_067_220526_M2_snap1.czi,33709.9,-13625,2022-05-26,09:34:44.1564688Z,4,1,1,1,1,1168,1168,VWF_067_220526_M2_snap1.czi,2.136709e-07,2.136709e-07,1.000000
1,VWF_067_220526_M2_snap2.czi,VWF_067_220526_M2_snap2.czi,34578.9,-12750.1,2022-05-26,10:06:15.2502142Z,4,1,1,1,1,1168,1168,VWF_067_220526_M2_snap2.czi,2.136709e-07,2.136709e-07,1.000000
2,VWF_067_220526_M2_snap3.czi,VWF_067_220526_M2_snap3.czi,33733.7,-13487.1,2022-05-26,10:09:26.3204531Z,4,1,1,1,1,1168,1168,VWF_067_220526_M2_snap3.czi,2.136709e-07,2.136709e-07,1.000000
3,VWF_067_220526_M2_snap4.czi,VWF_067_220526_M2_snap4.czi,33710.3,-13624.5,2022-05-26,10:14:39.7008037Z,4,1,1,1,1,1168,1168,VWF_067_220526_M2_snap4.czi,2.136709e-07,2.136709e-07,1.000000
4,VWF_067_220527_M1_snap1.czi,VWF_067_220527_M1_snap1.czi,2176.5,43712,2022-05-27,09:07:28.336385Z,4,1,1,1,1,1168,1168,VWF_067_220527_M1_snap1.czi,2.136709e-07,2.136709e-07,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,VWF_067_220527_M1_p8zstack2.czi_P8k,P8k,2720.6,43654.3,2022-05-27,10:00:15.0007039Z,4,1,8,1,1,533,513,VWF_067_220527_M1_p8zstack2.czi,4.874368e-07,4.874368e-07,0.000003
100,VWF_067_220527_M1_p8zstack2.czi_P9k,P9k,2731.1,43654.5,2022-05-27,10:00:15.0007039Z,4,1,8,1,1,533,513,VWF_067_220527_M1_p8zstack2.czi,4.874368e-07,4.874368e-07,0.000003
101,VWF_067_220527_M1_p8zstack2.czi_P11k,P11k,-99.5,44453.7,2022-05-27,10:00:15.0007039Z,4,1,8,1,1,533,513,VWF_067_220527_M1_p8zstack2.czi,4.874368e-07,4.874368e-07,0.000003
102,VWF_067_220527_M1_p8zstack2.czi_P19k2,P19k2,62.4,44524,2022-05-27,10:00:15.0007039Z,4,1,8,1,1,533,513,VWF_067_220527_M1_p8zstack2.czi,4.874368e-07,4.874368e-07,0.000003


In [14]:
pddfall=pd.concat([pddf, pddf1, pddf2, pddf3, pddf4, pddf5, pddf6, pddf7, pddf8, pddf9 ], axis =0)
pddfall
pddfall.to_csv('/Users/secchim/Documents/221102_VWF_043_045_048_049_053_060_062_063_066_067_IVM_metadata.csv', index=False) #only works for dataframes #otherwise you get row numbers


In [1]:
#METADATA OF OLD MICE


import os
import re
import json
import pandas as pd
import numpy as np
import czifile as zis

#empty dictionary
df={
    'Position_Name':[],
    'Cell_name':[],
    'Position_X':[],
    'Position_Y':[],
    'AcquisitionDate':[],
    'AcquisitionTime':[],
    'NChannel':[],
    'Nframes':[],
    'NZslices':[],
    'Tilerow':[],
    'Tilecolumn':[],
    'N_X':[],
    'N_Y':[],
    'Original_Filename':[],
    'Scale_X':[],
    'Scale_Y':[],
    'Scale_Z':[]
}



filepath='/Volumes/LoCelso/Secchi_Marine'

folder_list=['VWF_078_repeatedIVM_young_old' ]
#VWF_062_MS220418_IVMdepletion24h
remove_list=[]


for folder in folder_list:
    filepath=os.path.join(filepath,folder)

    for root, dirs, files in os.walk(filepath): # will open up all the folders, dirs is all the name of the folder it finds, files will contain all the filenames it finds
        for file in files:
            if file.endswith('.czi'):#need to be iterating through file
                duplicates=re.findall(".*(?=-LSM)",file)
                if len(duplicates)>1:
                    remove_list.append(duplicates[0]+'.czi') 
#                    print("test", file,duplicates[0])

        
        for file in files: 
            if file not in remove_list and file.endswith('.czi'):
                #print(file)
                filename=os.path.join(root,file)#looks at the directory used, joins the strings
                try:
                    czi = zis.CziFile(filename)
                except Exception as e:
                    print(f"{file} error {e}")
                    #print(filename)
                    # parse the XML into a dictionary
                metadatadict_czi = czi.metadata(raw=False)


                    

# For multiple positions per file wich should include movies and tilescans
                if "S" in metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys():
                    allscenes = metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['S']['Scenes']['Scene']
#                     if file == 'VWF_043_MS211125_m4-softwareautofocus_gfpref_1.czi':
#                         print(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image'].items())
                    if isinstance(allscenes, dict):
                        coordinates=allscenes['CenterPosition'].split(',')
                        df['Cell_name'].append(allscenes['Name'])
                        df['Position_Name'].append(file+'_'+allscenes['Name'])#changed from site to allscenes
                        df['Original_Filename'].append(file)
                        df['Position_X'].append(coordinates[0])
                        df['Position_Y'].append(coordinates[1])
                        dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                        df['NZslices'].append(dimensions['SizeZ']) 
                        df['NChannel'].append(dimensions['SizeC'])
                        df['Nframes'].append(dimensions['SizeT'])
                        df['N_X'].append(dimensions['SizeX'])
                        df['N_Y'].append(dimensions['SizeY'])
                        date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                        df['AcquisitionDate'].append(date[0])
                        df['AcquisitionTime'].append(date[1])
                        scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                        df['Scale_X'].append(scale['Value'])#Distance Id="X"
                        scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                        df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                        if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                            df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                        else:
                            df['Scale_Z'].append(1)#Distance Id="Z"

                        # try:
                        if "tilescan" in filename.casefold() and "RegionsSetup" in metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups'].keys(): 
                        #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            
                            tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                            if isinstance(tile, list):
                                for i in tile: #going inside the list !!!!
                                    #print(i)
                                    if i['IsUsedForAcquisition']==True:
                                        df['Tilecolumn'].append(i['Columns'])
                                        df['Tilerow'].append(i['Rows'])   
                                    else: 
                                        print(f'{file}IsUsedForAcquisition Not true')                                    

                            #will not work for single positions so the else is required
                            else:
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])
                
                        else:
                            df['Tilecolumn'].append('1')
                            df['Tilerow'].append('1')   
                        # except Exception as e:
                            # print(f"{file} error1 {e}")

                    else:
                            

                        for site in allscenes:#when scenes is a list you have to iterate through each element
                            #print(allscenes,file)
                            coordinates=site['CenterPosition'].split(',')
                            df['Cell_name'].append(site['Name'])
                            df['Position_Name'].append(file+'_'+site['Name'])
                            df['Original_Filename'].append(file)
                            df['Position_X'].append(coordinates[0])
                            df['Position_Y'].append(coordinates[1])
                            dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                            df['NZslices'].append(dimensions['SizeZ']) 
                            df['NChannel'].append(dimensions['SizeC'])
                            df['Nframes'].append(dimensions['SizeT'])
                            df['N_X'].append(dimensions['SizeX'])
                            df['N_Y'].append(dimensions['SizeY'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                            df['AcquisitionDate'].append(date[0])
                            df['AcquisitionTime'].append(date[1])
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                            df['Scale_X'].append(scale['Value'])#Distance Id="X"
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                            df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                            if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                                scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                                df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                            else:
                                df['Scale_Z'].append(1)#Distance Id="Z"

                            #try:
                            if "tilescan" in filename or "TILESCAN" in filename and "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                                tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])

                            else:
                                df['Tilecolumn'].append('1')
                                df['Tilerow'].append('1')   
                            #except Exception as e:
                            #    print(f"{file} error2")




#For single positions

                elif "HardwareSetting" in metadatadict_czi['ImageDocument']['Metadata'].keys() and "S" not in metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys():
                    
                    new_list=metadatadict_czi['ImageDocument']['Metadata']['HardwareSetting']['ParameterCollection']
                    for i in new_list: #i is every dict in the list 
#                         print(i)
                        if 'MTBStageAxisX' in i.values():
                            df['Cell_name'].append(file)
                            df['Position_Name'].append(file)
                            df['Original_Filename'].append(file)
#                             print(i['Position']['value'],i['Id'])
                            df['Position_X'].append(i['Position']['value'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                        if 'MTBStageAxisY' in i.values():
                            df['Position_Y'].append(i['Position']['value'])
                            dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                            df['NZslices'].append(dimensions['SizeZ']) 
                            df['NChannel'].append(dimensions['SizeC'])
                            df['Nframes'].append(dimensions['SizeT'])
                            df['N_X'].append(dimensions['SizeX'])
                            df['N_Y'].append(dimensions['SizeY'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                            df['AcquisitionDate'].append(date[0])
                            df['AcquisitionTime'].append(date[1])
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][0]
                            df['Scale_X'].append(scale['Value'])#Distance Id="X"
                            scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][1]
                            df['Scale_Y'].append(scale['Value'])#Distance Id="Y"
                            if len(metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'])>2:
                                scale=metadatadict_czi['ImageDocument']['Metadata']['Scaling']['Items']['Distance'][2]
                                df['Scale_Z'].append(scale['Value'])#Distance Id="Z"
                            else:
                                df['Scale_Z'].append(1)#Distance Id="Z"
                            #try:
                            if "tilescan" in filename or "TILESCAN" in filename and "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                                tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])

                            else:
                                df['Tilecolumn'].append('1')
                                df['Tilerow'].append('1')   
                        # except Exception as e:
                            #   print(f"{file} error3")

                    
                else:
                    print(file)
            
                

for key,value in df.items():
    print(key,len(df[key]))
pddf10=pd.DataFrame(df)
#pddf=pddf.drop_duplicates(subset=['Position_Name'])
#pddf
pddf10.to_csv('/Users/secchim/Documents/VWF_078_MS230210_metadata.csv', index=False) #only works for dataframes #otherwise you get row numbers

#pddf=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in df.items() ]))#fills empty with na
pddf10

VWF_078_MS230126_M6_tilescan_calvarium.cziIsUsedForAcquisition Not true
VWF_078_MS230126_M1_tilescan_calvarium.cziIsUsedForAcquisition Not true
VWF_078_MS230126_M1_tilescan_calvarium.cziIsUsedForAcquisition Not true
VWF_078_MS230126_M6_tilescan_calvarium-Stitching-01.cziIsUsedForAcquisition Not true
VWF_078_MS230126_M6_tilescan_calvarium-Stitching-01.cziIsUsedForAcquisition Not true
Position_Name 145
Cell_name 145
Position_X 145
Position_Y 145
AcquisitionDate 145
AcquisitionTime 145
NChannel 145
Nframes 145
NZslices 145
Tilerow 145
Tilecolumn 145
N_X 145
N_Y 145
Original_Filename 145
Scale_X 145
Scale_Y 145
Scale_Z 145


,Position_Name,Cell_name,Position_X,Position_Y,AcquisitionDate,AcquisitionTime,NChannel,Nframes,NZslices,Tilerow,Tilecolumn,N_X,N_Y,Original_Filename,Scale_X,Scale_Y,Scale_Z
0,VWF_078_MS230119_M1_tilescan_Day0.czi_TR1,TR1,1926.81,43703.35,2023-01-19,10:53:20.7861624Z,4,1,31,4,7,3278,1894,VWF_078_MS230119_M1_tilescan_Day0.czi,8.286426e-07,8.286426e-07,0.000003
1,VWF_078_MS230119_M3_tilescan_Day0.czi_TR1,TR1,1933.846,44151.827,2023-01-19,12:44:49.4103127Z,4,1,49,4,6,2818,1895,VWF_078_MS230119_M3_tilescan_Day0.czi,8.286426e-07,8.286426e-07,0.000003
2,VWF_078_MS230119_M2_tilescan_Day0.czi_TR1,TR1,2463.176,43972.248,2023-01-19,14:51:13.9220218Z,4,1,39,4,6,2818,1895,VWF_078_MS230119_M2_tilescan_Day0.czi,8.286426e-07,8.286426e-07,0.000003
3,VWF_078_MS230119_M4_tilescan_Day0.czi_TR1,TR1,225.433,45243.897,2023-01-19,16:29:28.786703Z,4,1,67,4,6,2818,1894,VWF_078_MS230119_M4_tilescan_Day0.czi,8.286426e-07,8.286426e-07,0.000003
4,VWF_078_MS230120_M1_tilescan_Day1.czi_TR1,TR1,2751.271,43473.337,2023-01-20,09:36:45.4986237Z,4,1,35,4,7,3278,1895,VWF_078_MS230120_M1_tilescan_Day1.czi,8.286426e-07,8.286426e-07,0.000003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,VWF_078_MS230120_M3_tilescan_Day1-Stitching-02...,TR1,2680.64,43812.036,2023-01-20,11:33:41.287475Z,4,1,49,4,6,2818,1862,VWF_078_MS230120_M3_tilescan_Day1-Stitching-02...,8.286426e-07,8.286426e-07,0.000003
141,VWF_078_MS230122_M5_tilescan_Day0-Stitching-02...,TR1,3097.13,43544.415,2023-01-22,08:35:54.7027833Z,4,1,60,4,6,2777,1921,VWF_078_MS230122_M5_tilescan_Day0-Stitching-02...,8.286426e-07,8.286426e-07,0.000003
142,VWF_078_MS230123_M5_tilescan_Day1-Stitching-05...,TR1,3223.151,43581.755,2023-01-23,09:35:39.0813011Z,4,1,60,4,6,2824,1904,VWF_078_MS230123_M5_tilescan_Day1-Stitching-05...,8.286426e-07,8.286426e-07,0.000003
143,VWF_078_MS230122_M6_tilescan_Day0-Stitching-03...,TR1,3330.795,44577.237,2023-01-22,10:17:33.4693497Z,4,1,42,4,6,2821,1901,VWF_078_MS230122_M6_tilescan_Day0-Stitching-03...,8.286426e-07,8.286426e-07,0.000003
